# PMEmo Dataset - Preprocessing

# Done on local computer

## Import relevant libraries

In [1]:
import numpy as np
import ast
import pandas as pd
import matplotlib.pyplot as plt

import os

Set file directory variables

In [2]:
import sys
sys.path.insert(1, '../../utils')
from paths import *

## Process the annotation dataframes

Read static song-level annotations

In [3]:
df_annotations = pd.read_csv(get_pmemo_path('annotations/static_annotations.csv'))
display(df_annotations)
print(df_annotations.shape)

,musicId,Arousal(mean),Valence(mean)
0,1,0.4000,0.5750
1,4,0.2625,0.2875
2,5,0.1500,0.2000
3,6,0.5125,0.3500
4,7,0.7000,0.7250
...,...,...,...
762,993,0.8625,0.7625
763,996,0.8750,0.5625
764,997,0.7125,0.6625
765,999,0.8750,0.7750


(767, 3)


In [4]:
df_annotations = df_annotations.rename(columns={'musicId': 'song_id'})
df_annotations

,song_id,Arousal(mean),Valence(mean)
0,1,0.4000,0.5750
1,4,0.2625,0.2875
2,5,0.1500,0.2000
3,6,0.5125,0.3500
4,7,0.7000,0.7250
...,...,...,...
762,993,0.8625,0.7625
763,996,0.8750,0.5625
764,997,0.7125,0.6625
765,999,0.8750,0.7750


In [5]:
df_annotations.describe()

,song_id,Arousal(mean),Valence(mean)
count,767.000000,767.000000,767.000000
mean,493.770535,0.622355,0.596581
std,285.958166,0.184812,0.161986
min,1.000000,0.087500,0.125000
25%,238.500000,0.500000,0.487500
50%,495.000000,0.650000,0.625000
75%,739.500000,0.762500,0.725000
max,1000.000000,0.975000,0.912500


Map the valence and arousal values in the dataset, ranging from 1 to 9, to values ranging from -1 to 1, to follow convention

In [6]:
def map_va_value(value):
  old_min = 0
  old_max = 1

  new_min = -1
  new_max = 1

  mapped_value = ((value - old_min) * (new_max - new_min) / (old_max - old_min)) + new_min
  return mapped_value

# Test the function
for i in range(0, 2):
    print(f"Original value: {i}, Mapped value: {map_va_value(i)}")

Original value: 0, Mapped value: -1.0
Original value: 1, Mapped value: 1.0


In [7]:
df_annotations['valence_mean_mapped'] = df_annotations['Valence(mean)'].apply(map_va_value)
df_annotations['arousal_mean_mapped'] = df_annotations['Arousal(mean)'].apply(map_va_value)
df_annotations = df_annotations.drop(['Valence(mean)', 'Arousal(mean)',], axis=1)

In [8]:
df_annotations

,song_id,valence_mean_mapped,arousal_mean_mapped
0,1,0.150,-0.200
1,4,-0.425,-0.475
2,5,-0.600,-0.700
3,6,-0.300,0.025
4,7,0.450,0.400
...,...,...,...
762,993,0.525,0.725
763,996,0.125,0.750
764,997,0.325,0.425
765,999,0.550,0.750


Export the dataframe

In [9]:
df_annotations.to_csv(get_pmemo_path('processed/annotations/pmemo_static_annotations.csv'), index=False)

Import the dataframe

In [10]:
df_annotations = pd.read_csv(get_pmemo_path('processed/annotations/pmemo_static_annotations.csv'))
df_annotations

,song_id,valence_mean_mapped,arousal_mean_mapped
0,1,0.150,-0.200
1,4,-0.425,-0.475
2,5,-0.600,-0.700
3,6,-0.300,0.025
4,7,0.450,0.400
...,...,...,...
762,993,0.525,0.725
763,996,0.125,0.750
764,997,0.325,0.425
765,999,0.550,0.750


## Process the Essentia features datasets

Define function to check for any non-float/int columns and deal with them

In [11]:
df_essentia_features = pd.read_csv(get_pmemo_path('processed/features/essentia_features.csv'))
df_essentia_features

,Unnamed: 0,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,...,metadata.version.essentia_git_sha,metadata.version.extractor,tonal.chords_key,tonal.chords_scale,tonal.key_edma.key,tonal.key_edma.scale,tonal.key_krumhansl.key,tonal.key_krumhansl.scale,tonal.key_temperley.key,tonal.key_temperley.scale
0,0,1,0.970365,2.386482,3.594657,6.740679,14.444414,25.484644,11.411017,10.707484,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,G,major,G,major,G,major,G,major
1,1,4,0.938222,2.208391,3.674144,5.080004,13.383171,24.118259,14.166083,14.635095,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,F#,major,Bb,major,Bb,major,Bb,major
2,2,5,0.930816,2.393671,3.616534,9.753744,20.003422,26.715527,18.050819,18.184444,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,A,minor,D,minor,D,minor,D,minor
3,3,6,0.954429,1.709188,2.437101,4.900128,8.548371,24.963417,17.043421,18.244125,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,E,minor,C,major,C,major,C,major
4,4,7,0.970629,3.049778,4.716875,10.767557,23.465727,25.671312,10.193353,8.912777,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,G,major,G,major,G,major,G,major
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,789,993,0.900841,2.860650,4.524299,7.774405,19.612854,24.813204,11.216174,10.689340,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,F,minor,Ab,major,Ab,major,Ab,major
790,790,996,0.923707,2.780330,4.445892,9.029428,20.219875,25.388409,10.426634,8.341273,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,C#,minor,C#,minor,C#,minor,C#,minor
791,791,997,0.913454,2.712144,4.339530,7.626777,20.528904,25.967821,11.849738,11.506009,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,Eb,major,Ab,major,Ab,major,Ab,major
792,792,999,0.955402,2.754516,3.943231,5.839927,11.535942,25.688959,11.403626,9.805414,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,B,minor,B,minor,B,minor,B,minor


Drop the 'Unnamed: 0	' column

In [12]:
# drop Unnamed:0 column
df_essentia_features = df_essentia_features[df_essentia_features.columns[1:]]
df_essentia_features

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,metadata.version.essentia_git_sha,metadata.version.extractor,tonal.chords_key,tonal.chords_scale,tonal.key_edma.key,tonal.key_edma.scale,tonal.key_krumhansl.key,tonal.key_krumhansl.scale,tonal.key_temperley.key,tonal.key_temperley.scale
0,1,0.970365,2.386482,3.594657,6.740679,14.444414,25.484644,11.411017,10.707484,2.498400,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,G,major,G,major,G,major,G,major
1,4,0.938222,2.208391,3.674144,5.080004,13.383171,24.118259,14.166083,14.635095,4.408802,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,F#,major,Bb,major,Bb,major,Bb,major
2,5,0.930816,2.393671,3.616534,9.753744,20.003422,26.715527,18.050819,18.184444,2.864332,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,A,minor,D,minor,D,minor,D,minor
3,6,0.954429,1.709188,2.437101,4.900128,8.548371,24.963417,17.043421,18.244125,2.253785,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,E,minor,C,major,C,major,C,major
4,7,0.970629,3.049778,4.716875,10.767557,23.465727,25.671312,10.193353,8.912777,2.406512,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,G,major,G,major,G,major,G,major
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.900841,2.860650,4.524299,7.774405,19.612854,24.813204,11.216174,10.689340,2.157647,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,F,minor,Ab,major,Ab,major,Ab,major
790,996,0.923707,2.780330,4.445892,9.029428,20.219875,25.388409,10.426634,8.341273,2.059642,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,C#,minor,C#,minor,C#,minor,C#,minor
791,997,0.913454,2.712144,4.339530,7.626777,20.528904,25.967821,11.849738,11.506009,2.347258,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,Eb,major,Ab,major,Ab,major,Ab,major
792,999,0.955402,2.754516,3.943231,5.839927,11.535942,25.688959,11.403626,9.805414,2.655332,...,v2.1_beta5-1110-g77a6a954-dirty,music 2.0,B,minor,B,minor,B,minor,B,minor


See what features are available

In [13]:
print(df_essentia_features.columns.to_list())

['song_id', 'lowlevel.average_loudness', 'lowlevel.barkbands_crest.dmean', 'lowlevel.barkbands_crest.dmean2', 'lowlevel.barkbands_crest.dvar', 'lowlevel.barkbands_crest.dvar2', 'lowlevel.barkbands_crest.max', 'lowlevel.barkbands_crest.mean', 'lowlevel.barkbands_crest.median', 'lowlevel.barkbands_crest.min', 'lowlevel.barkbands_crest.stdev', 'lowlevel.barkbands_crest.var', 'lowlevel.barkbands_flatness_db.dmean', 'lowlevel.barkbands_flatness_db.dmean2', 'lowlevel.barkbands_flatness_db.dvar', 'lowlevel.barkbands_flatness_db.dvar2', 'lowlevel.barkbands_flatness_db.max', 'lowlevel.barkbands_flatness_db.mean', 'lowlevel.barkbands_flatness_db.median', 'lowlevel.barkbands_flatness_db.min', 'lowlevel.barkbands_flatness_db.stdev', 'lowlevel.barkbands_flatness_db.var', 'lowlevel.barkbands_kurtosis.dmean', 'lowlevel.barkbands_kurtosis.dmean2', 'lowlevel.barkbands_kurtosis.dvar', 'lowlevel.barkbands_kurtosis.dvar2', 'lowlevel.barkbands_kurtosis.max', 'lowlevel.barkbands_kurtosis.mean', 'lowlevel.ba

Get song_ids

In [14]:
song_ids = df_essentia_features['song_id'].values.tolist()
print(song_ids)

[1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 80, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 113, 114, 115, 116, 117, 118, 119, 120, 123, 124, 126, 127, 128, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 156, 157, 158, 159, 160, 161, 162, 165, 166, 167, 168, 169, 170, 171, 172, 174, 175, 176, 177, 179, 180, 181, 184, 185, 186, 187, 188, 189, 191, 192, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 209, 210, 211, 212, 213, 215, 216, 217, 219, 220, 221, 222, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 241, 242, 244, 249, 251, 252, 253, 257, 258, 260, 263, 264, 265, 267, 269, 273

Some features are irrelevant, or are metadata. Drop them

In [15]:
metadata_columns = [col for col in df_essentia_features.columns if 'metadata' in col]
df_essentia_features = df_essentia_features.drop(columns=metadata_columns)
df_essentia_features

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,lowlevel.mfcc.cov,lowlevel.mfcc.icov,tonal.chords_key,tonal.chords_scale,tonal.key_edma.key,tonal.key_edma.scale,tonal.key_krumhansl.key,tonal.key_krumhansl.scale,tonal.key_temperley.key,tonal.key_temperley.scale
0,1,0.970365,2.386482,3.594657,6.740679,14.444414,25.484644,11.411017,10.707484,2.498400,...,[[ 6.85665918e+03 -2.10462231e+03 4.07335205e...,[[ 2.3996759e-04 1.5186716e-04 -1.0741219e-04...,G,major,G,major,G,major,G,major
1,4,0.938222,2.208391,3.674144,5.080004,13.383171,24.118259,14.166083,14.635095,4.408802,...,[[ 1.05960488e+04 -2.46355591e+03 -3.73934814e...,[[ 2.27190103e-04 2.40043330e-04 2.74839957e...,F#,major,Bb,major,Bb,major,Bb,major
2,5,0.930816,2.393671,3.616534,9.753744,20.003422,26.715527,18.050819,18.184444,2.864332,...,[[ 9.01584277e+03 -2.88903076e+03 -2.05213814e...,[[ 1.6499990e-04 1.4211204e-04 8.3164217e-05...,A,minor,D,minor,D,minor,D,minor
3,6,0.954429,1.709188,2.437101,4.900128,8.548371,24.963417,17.043421,18.244125,2.253785,...,[[ 1.82411934e+04 -3.36026880e+03 -7.82240356e...,[[ 1.13304508e-04 1.76178699e-04 1.48005391e...,E,minor,C,major,C,major,C,major
4,7,0.970629,3.049778,4.716875,10.767557,23.465727,25.671312,10.193353,8.912777,2.406512,...,[[ 4.75388525e+03 -1.74141968e+02 -9.13416977e...,[[ 2.7016792e-04 9.4603121e-05 1.3104909e-04...,G,major,G,major,G,major,G,major
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.900841,2.860650,4.524299,7.774405,19.612854,24.813204,11.216174,10.689340,2.157647,...,[[ 7.2658301e+03 -8.4079852e+02 -3.8062350e+02...,[[ 1.8072149e-04 5.5549688e-05 3.2895543e-05...,F,minor,Ab,major,Ab,major,Ab,major
790,996,0.923707,2.780330,4.445892,9.029428,20.219875,25.388409,10.426634,8.341273,2.059642,...,[[17220.953 41.737896 -326.00067 -203...,[[ 6.49392823e-05 -1.75009172e-05 5.02493858e...,C#,minor,C#,minor,C#,minor,C#,minor
791,997,0.913454,2.712144,4.339530,7.626777,20.528904,25.967821,11.849738,11.506009,2.347258,...,[[ 3.15097598e+04 2.41078491e+03 5.18551941e...,[[ 4.48296960e-05 -5.64669099e-05 -2.36702836e...,Eb,major,Ab,major,Ab,major,Ab,major
792,999,0.955402,2.754516,3.943231,5.839927,11.535942,25.688959,11.403626,9.805414,2.655332,...,[[ 5.7085840e+04 3.7435229e+03 3.6263333e+03...,[[ 4.5769535e-05 -1.8094525e-04 -2.0284149e-05...,B,minor,B,minor,B,minor,B,minor


Find out if any columns are not of the type float or int

In [16]:
pd.set_option('display.max_columns', None)
df_essentia_features.select_dtypes(exclude=['int64', 'float64'])

,lowlevel.barkbands.dmean,lowlevel.barkbands.dmean2,lowlevel.barkbands.dvar,lowlevel.barkbands.dvar2,lowlevel.barkbands.max,lowlevel.barkbands.mean,lowlevel.barkbands.median,lowlevel.barkbands.min,lowlevel.barkbands.stdev,lowlevel.barkbands.var,lowlevel.erbbands.dmean,lowlevel.erbbands.dmean2,lowlevel.erbbands.dvar,lowlevel.erbbands.dvar2,lowlevel.erbbands.max,lowlevel.erbbands.mean,lowlevel.erbbands.median,lowlevel.erbbands.min,lowlevel.erbbands.stdev,lowlevel.erbbands.var,lowlevel.gfcc.mean,lowlevel.melbands.dmean,lowlevel.melbands.dmean2,lowlevel.melbands.dvar,lowlevel.melbands.dvar2,lowlevel.melbands.max,lowlevel.melbands.mean,lowlevel.melbands.median,lowlevel.melbands.min,lowlevel.melbands.stdev,lowlevel.melbands.var,lowlevel.melbands128.dmean,lowlevel.melbands128.dmean2,lowlevel.melbands128.dvar,lowlevel.melbands128.dvar2,lowlevel.melbands128.max,lowlevel.melbands128.mean,lowlevel.melbands128.median,lowlevel.melbands128.min,lowlevel.melbands128.stdev,lowlevel.melbands128.var,lowlevel.mfcc.mean,lowlevel.spectral_contrast_coeffs.dmean,lowlevel.spectral_contrast_coeffs.dmean2,lowlevel.spectral_contrast_coeffs.dvar,lowlevel.spectral_contrast_coeffs.dvar2,lowlevel.spectral_contrast_coeffs.max,lowlevel.spectral_contrast_coeffs.mean,lowlevel.spectral_contrast_coeffs.median,lowlevel.spectral_contrast_coeffs.min,lowlevel.spectral_contrast_coeffs.stdev,lowlevel.spectral_contrast_coeffs.var,lowlevel.spectral_contrast_valleys.dmean,lowlevel.spectral_contrast_valleys.dmean2,lowlevel.spectral_contrast_valleys.dvar,lowlevel.spectral_contrast_valleys.dvar2,lowlevel.spectral_contrast_valleys.max,lowlevel.spectral_contrast_valleys.mean,lowlevel.spectral_contrast_valleys.median,lowlevel.spectral_contrast_valleys.min,lowlevel.spectral_contrast_valleys.stdev,lowlevel.spectral_contrast_valleys.var,rhythm.beats_loudness_band_ratio.dmean,rhythm.beats_loudness_band_ratio.dmean2,rhythm.beats_loudness_band_ratio.dvar,rhythm.beats_loudness_band_ratio.dvar2,rhythm.beats_loudness_band_ratio.max,rhythm.beats_loudness_band_ratio.mean,rhythm.beats_loudness_band_ratio.median,rhythm.beats_loudness_band_ratio.min,rhythm.beats_loudness_band_ratio.stdev,rhythm.beats_loudness_band_ratio.var,tonal.hpcp.dmean,tonal.hpcp.dmean2,tonal.hpcp.dvar,tonal.hpcp.dvar2,tonal.hpcp.max,tonal.hpcp.mean,tonal.hpcp.median,tonal.hpcp.min,tonal.hpcp.stdev,tonal.hpcp.var,rhythm.beats_position,rhythm.bpm_histogram,tonal.chords_histogram,tonal.thpcp,lowlevel.gfcc.cov,lowlevel.gfcc.icov,lowlevel.mfcc.cov,lowlevel.mfcc.icov,tonal.chords_key,tonal.chords_scale,tonal.key_edma.key,tonal.key_edma.scale,tonal.key_krumhansl.key,tonal.key_krumhansl.scale,tonal.key_temperley.key,tonal.key_temperley.scale
0,[6.7507481e-04 4.9614939e-03 1.3591013e-03 1.2...,[1.0240980e-03 7.5850049e-03 2.5296274e-03 2.4...,[1.7045637e-06 2.5856021e-04 4.7740188e-05 4.0...,[3.25299357e-06 5.83966437e-04 1.37791023e-04 ...,[0.01999433 0.13475272 0.06416687 0.04995539 0...,[3.0178686e-03 1.1141291e-02 8.9332712e-04 7.7...,[1.7186446e-03 6.0509765e-03 6.4177446e-05 4.1...,[2.2496570e-24 1.7837038e-23 1.7201074e-23 4.8...,[3.5796694e-03 1.9511294e-02 5.1862756e-03 4.6...,[1.2814033e-05 3.8069056e-04 2.6897455e-05 2.1...,[2.16701984e-01 9.00710642e-01 1.35087109e+00 ...,[3.2208666e-01 1.3940675e+00 2.4691632e+00 6.2...,[4.6072030e-01 9.8126659e+00 4.2747158e+01 2.9...,[9.6904051e-01 2.3564703e+01 1.1948914e+02 8.6...,[6.7880836e+00 2.8372843e+01 5.9488792e+01 1.3...,[6.0949785e-01 1.3509623e+00 1.1622906e+00 2.0...,[3.78775269e-01 4.30995047e-01 1.54191434e-01 ...,[6.17754173e-22 3.40790617e-21 1.86307570e-20 ...,[8.8675076e-01 3.4459212e+00 5.2156372e+00 1.2...,[7.8632694e-01 1.1874373e+01 2.7202873e+01 1.5...,[ -18.483932 65.78962 -126.755196 26.2...,[1.61825970e-03 7.41391443e-04 5.61211491e-04 ...,[2.41026189e-03 1.18709146e-03 1.07337942e-03 ...,[2.56003004e-05 8.61925309e-06 8.24445760e-06 ...,[5.5773689e-05 2.1885809e-05 2.4230025e-05 1.7...,[0.04665443 0.0284497 0.02166686 0.00605062 0...,[4.2022024e-03 9.2

Seems like there are 3 main types of column types:

1. ndarray
2. string
3. float64 or int64

Get the columns whose type is ndarray

In [17]:
pd.reset_option('display.max_columns')

string_columns = ['tonal.chords_key',
                  'tonal.chords_scale',
                  'tonal.key_edma.key',
                  'tonal.key_edma.scale',
                  'tonal.key_krumhansl.key',
                  'tonal.key_krumhansl.scale',
                  'tonal.key_temperley.key',
                  'tonal.key_temperley.scale'
                  ]

df_essentia_features_ndarray_columns = df_essentia_features.select_dtypes(exclude=['int64', 'float64'])
ndarray_columns = df_essentia_features_ndarray_columns.columns.difference(string_columns)
df_essentia_features_ndarray_columns = df_essentia_features_ndarray_columns[ndarray_columns]
df_essentia_features_ndarray_columns

,lowlevel.barkbands.dmean,lowlevel.barkbands.dmean2,lowlevel.barkbands.dvar,lowlevel.barkbands.dvar2,lowlevel.barkbands.max,lowlevel.barkbands.mean,lowlevel.barkbands.median,lowlevel.barkbands.min,lowlevel.barkbands.stdev,lowlevel.barkbands.var,...,tonal.hpcp.dmean2,tonal.hpcp.dvar,tonal.hpcp.dvar2,tonal.hpcp.max,tonal.hpcp.mean,tonal.hpcp.median,tonal.hpcp.min,tonal.hpcp.stdev,tonal.hpcp.var,tonal.thpcp
0,[6.7507481e-04 4.9614939e-03 1.3591013e-03 1.2...,[1.0240980e-03 7.5850049e-03 2.5296274e-03 2.4...,[1.7045637e-06 2.5856021e-04 4.7740188e-05 4.0...,[3.25299357e-06 5.83966437e-04 1.37791023e-04 ...,[0.01999433 0.13475272 0.06416687 0.04995539 0...,[3.0178686e-03 1.1141291e-02 8.9332712e-04 7.7...,[1.7186446e-03 6.0509765e-03 6.4177446e-05 4.1...,[2.2496570e-24 1.7837038e-23 1.7201074e-23 4.8...,[3.5796694e-03 1.9511294e-02 5.1862756e-03 4.6...,[1.2814033e-05 3.8069056e-04 2.6897455e-05 2.1...,...,[0.17198518 0.20222759 0.16056684 0.08428085 0...,[0.03722104 0.05006677 0.03816758 0.01647517 0...,[0.10280088 0.12868041 0.10828324 0.04565451 0...,[1. 1. 1. 1. 1...,[0.08938619 0.11562601 0.0685273 0.02887096 0...,[0.01410414 0.0140431 0.00838527 0.00190963 0...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0.19092269 0.23774686 0.16630177 0.10306383 0...,[0.03645147 0.05652357 0.02765628 0.01062215 0...,[1. 0.43556923 0.25200686 0.19656383 0...
1,[2.1251652e-02 1.7291097e-02 3.3005469e-03 8.9...,[3.4744766e-02 2.6315041e-02 6.3065514e-03 1.6...,[6.3025439e-04 2.4410433e-03 2.6475094e-04 1.8...,[1.40089623e-03 5.08765550e-03 7.77634385e-04 ...,[2.9968530e-01 5.1625466e-01 1.3678691e-01 4.7...,[8.2245916e-02 7.5655505e-02 2.1220266e-03 6.7...,[6.7539424e-02 4.8154771e-02 3.0250652e-04 1.6...,[2.99164139e-23 1.80910891e-23 2.63501725e-23 ...,[7.0648007e-02 8.8403553e-02 1.1909845e-02 3.2...,[4.9911407e-03 7.8151878e-03 1.4184442e-04 1.0...,...,[0.03601822 0.02717334 0.13093984 0.08780509 0...,[0.00820964 0.00508901 0.01938181 0.02851443 0...,[0.01872263 0.01395738 0.05156266 0.04790737 0...,[1. 0.8865714 1. 1. 1...,[0.01961876 0.01111388 0.16504459 0.38964278 0...,[0.00095513 0.00053815 0.00351087 0.01251633 0...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0.09778243 0.05829102 0.24579811 0.4666808 0...,[0.0095614 0.00339784 0.06041671 0.21779096 0...,[1. 0.7764151 0.30383804 0.39174917 0...
2,[2.0660346e-03 1.7046761e-02 4.3932800e-03 1.9...,[3.0170975e-03 2.3500836e-02 8.1736939e-03 3.6...,[2.4459156e-05 1.8362320e-03 2.9854872e-04 6.2...,[4.9572885e-05 3.4178055e-03 8.5638970e-04 1.8...,[7.2549619e-02 4.7484457e-01 1.6325498e-01 8.8...,[8.8138906e-03 7.6573014e-02 3.1388358e-03 1.5...,[2.2026242e-03 4.7468882e-02 3.9559673e-04 3.4...,[4.0490242e-23 3.8137594e-23 2.5247948e-23 2.0...,[1.42243495e-02 8.76345634e-02 1.31846657e-02 ...,[2.02332129e-04 7.67981680e-03 1.73835404e-04 ...,...,[0.24058501 0.21798588 0.2825976 0.15549487 0...,[0.04838407 0.0670328 0.0516134 0.04138399 0...,[0.12234772 0.15207846 0.1398726 0.11865796 0...,[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. ...,[0.36658412 0.49308702 0.19638392 0.06267487 0...,[0.15527296 0.30562785 0.16246429 0.00461627 0...,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[0.37299344 0.46632305 0.22388871 0.1771584 0...,[0.13912411 0.21745718 0.05012615 0.0313851 0...,[1. 0.39827436 0.12710713 0.06779694 0...
3,[1.0009357e-03 4.1595874e-03 1.2349858e-03 6.2...,[1.6989100e-03 6.4974395e-03 2.0641894e-03 1.1...,[2.7092769e-06 8.4232619e-05 2.1404236e-05 5.6...,[6.0517355e-06 1.9099977e-04 5.2544587e-05 1.6...,[2.3652460e-02 1.3755780e-01 4.6066619e-02 2.8...,[6.3011521e-03 3.6585730e-02 1.9028852e-03 1.0...,[5.3758118e-03 4.6566591e-02 8.6422917e-04 5.3...,[4.2340860e-23 2.8940151e-23 1.7776786e-23 1.3...,[5.2100033e-03 2.3416653e-02 4.5262915e-03 2.0...,[2.7144137e-05 5.4833968e-04 2.0487316e-05 4.1...,...,[0.08754273 0.07279804 0.06281674 0.06875471 0...,[0.02282552 0.0164335 0.01370707 0.01573125 0...,[0.06398014 0.04620796 0.0389328 0.04402793 0...,[1. 1. 1. 1.

Print one random value

In [18]:
print(df_essentia_features_ndarray_columns['lowlevel.mfcc.icov'][0])

[[ 2.3996759e-04  1.5186716e-04 -1.0741219e-04 -8.6460122e-06
   2.2302331e-04  9.4255091e-05  5.3184431e-05  1.8742828e-04
  -2.1787897e-04 -3.1957857e-04 -1.9685674e-04 -2.0849690e-04
   1.1779139e-04]
 [ 1.5186716e-04  6.0764723e-04  3.0488841e-04  2.9659504e-04
  -2.3486497e-04 -2.3912815e-04 -1.8182107e-04 -2.3828326e-04
  -2.8617471e-04 -9.7164426e-05 -3.2422446e-05  1.1898611e-04
  -4.0961319e-04]
 [-1.0741219e-04  3.0488841e-04  1.8638392e-03 -2.7468003e-04
  -6.5738900e-04 -7.5130764e-04 -5.4159737e-04 -7.7709946e-04
   4.0957029e-04  3.1211090e-04  6.4047013e-04  1.1044028e-04
  -7.2906312e-04]
 [-8.6460122e-06  2.9659504e-04 -2.7468003e-04  3.7150935e-03
  -1.2925579e-03 -7.6804939e-04  2.1459775e-04 -7.4678764e-04
  -9.2790951e-04 -7.5283402e-04  1.1883125e-03 -3.8307830e-04
  -1.1139122e-03]
 [ 2.2302331e-04 -2.3486497e-04 -6.5738900e-04 -1.2925579e-03
   3.6607434e-03 -6.0802820e-04  1.2735978e-03  8.3780737e-04
  -3.5398774e-04 -1.1457135e-03 -1.2168454e-03  6.4217886e-0

Flatten the columns whose values are ndarrays, like tonal.chords_histogram

Credits to https://stackoverflow.com/questions/45704999/how-to-convert-vector-wrapped-as-string-to-numpy-array-in-pandas-dataframe

In [19]:
def string_to_ndarray(str):
  return np.fromstring(str.replace('\n','')
                       .replace('[','')
                       .replace(']','')
                       .replace('  ',' '), 
                       sep=' '
                       )

In [20]:
df_essentia_features_ndarray_columns = df_essentia_features_ndarray_columns.applymap(string_to_ndarray)
df_essentia_features_ndarray_columns

/var/folders/ft/gzvmkgdj727g157scc1xw9f80000gn/T/ipykernel_66615/3672827613.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_essentia_features_ndarray_columns = df_essentia_features_ndarray_columns.applymap(string_to_ndarray)


,lowlevel.barkbands.dmean,lowlevel.barkbands.dmean2,lowlevel.barkbands.dvar,lowlevel.barkbands.dvar2,lowlevel.barkbands.max,lowlevel.barkbands.mean,lowlevel.barkbands.median,lowlevel.barkbands.min,lowlevel.barkbands.stdev,lowlevel.barkbands.var,...,tonal.hpcp.dmean2,tonal.hpcp.dvar,tonal.hpcp.dvar2,tonal.hpcp.max,tonal.hpcp.mean,tonal.hpcp.median,tonal.hpcp.min,tonal.hpcp.stdev,tonal.hpcp.var,tonal.thpcp
0,"[0.00067507481, 0.0049614939, 0.0013591013, 0....","[0.001024098, 0.0075850049, 0.0025296274, 0.00...","[1.7045637e-06, 0.00025856021, 4.7740188e-05, ...","[3.25299357e-06, 0.000583966437, 0.00013779102...","[0.01999433, 0.13475272, 0.06416687, 0.0499553...","[0.0030178686, 0.011141291, 0.00089332712, 0.0...","[0.0017186446, 0.0060509765, 6.4177446e-05, 4....","[2.249657e-24, 1.7837038e-23, 1.7201074e-23, 4...","[0.0035796694, 0.019511294, 0.0051862756, 0.00...","[1.2814033e-05, 0.00038069056, 2.6897455e-05, ...",...,"[0.17198518, 0.20222759, 0.16056684, 0.0842808...","[0.03722104, 0.05006677, 0.03816758, 0.0164751...","[0.10280088, 0.12868041, 0.10828324, 0.0456545...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.08938619, 0.11562601, 0.0685273, 0.02887096...","[0.01410414, 0.0140431, 0.00838527, 0.00190963...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.19092269, 0.23774686, 0.16630177, 0.1030638...","[0.03645147, 0.05652357, 0.02765628, 0.0106221...","[1.0, 0.43556923, 0.25200686, 0.19656383, 0.15..."
1,"[0.021251652, 0.017291097, 0.0033005469, 0.000...","[0.034744766, 0.026315041, 0.0063065514, 0.001...","[0.00063025439, 0.0024410433, 0.00026475094, 1...","[0.00140089623, 0.0050876555, 0.000777634385, ...","[0.2996853, 0.51625466, 0.13678691, 0.04799056...","[0.082245916, 0.075655505, 0.0021220266, 0.000...","[0.067539424, 0.048154771, 0.00030250652, 0.00...","[2.99164139e-23, 1.80910891e-23, 2.63501725e-2...","[0.070648007, 0.088403553, 0.011909845, 0.0032...","[0.0049911407, 0.0078151878, 0.00014184442, 1....",...,"[0.03601822, 0.02717334, 0.13093984, 0.0878050...","[0.00820964, 0.00508901, 0.01938181, 0.0285144...","[0.01872263, 0.01395738, 0.05156266, 0.0479073...","[1.0, 0.8865714, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[0.01961876, 0.01111388, 0.16504459, 0.3896427...","[0.00095513, 0.00053815, 0.00351087, 0.0125163...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.09778243, 0.05829102, 0.24579811, 0.4666808...","[0.0095614, 0.00339784, 0.06041671, 0.21779096...","[1.0, 0.7764151, 0.30383804, 0.39174917, 0.413..."
2,"[0.0020660346, 0.017046761, 0.00439328, 0.0019...","[0.0030170975, 0.023500836, 0.0081736939, 0.00...","[2.4459156e-05, 0.001836232, 0.00029854872, 6....","[4.9572885e-05, 0.0034178055, 0.0008563897, 0....","[0.072549619, 0.47484457, 0.16325498, 0.088804...","[0.0088138906, 0.076573014, 0.0031388358, 0.00...","[0.0022026242, 0.047468882, 0.00039559673, 0.0...","[4.0490242e-23, 3.8137594e-23, 2.5247948e-23, ...","[0.0142243495, 0.0876345634, 0.0131846657, 0.0...","[0.000202332129, 0.0076798168, 0.000173835404,...",...,"[0.24058501, 0.21798588, 0.2825976, 0.15549487...","[0.04838407, 0.0670328, 0.0516134, 0.04138399,...","[0.12234772, 0.15207846, 0.1398726, 0.11865796...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.36658412, 0.49308702, 0.19638392, 0.0626748...","[0.15527296, 0.30562785, 0.16246429, 0.0046162...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.37299344, 0.46632305, 0.22388871, 0.1771584...","[0.13912411, 0.21745718, 0.05012615, 0.0313851...","[1.0, 0.39827436, 0.12710713, 0.06779694, 0.06..."
3,"[0.0010009357, 0.0041595874, 0.0012349858, 0.0...","[0.00169891, 0.0064974395, 0.0020641894, 0.001...","[2.7092769e-06, 8.4232619e-05, 2.1404236e-05, ...","[6.0517355e-06, 0.00019099977, 5.2544587e-05, ...","[0.02365246, 0.1375578, 0.046066619, 0.0285861...","[0.0063011521, 0.03658573, 0.0019028852, 0.001...","[0.0053758118, 0.046566591, 0.00086422917, 0.0...","[4.234086e-23, 2.8940151e-23, 1.7776786e-23, 1...","[0.0052100033, 0.023416653, 0.00452

Print out that same value to ensure that it is properly converted to a ndarray

In [21]:
print(df_essentia_features_ndarray_columns['lowlevel.mfcc.icov'][0])
print(type(df_essentia_features_ndarray_columns['lowlevel.mfcc.icov'][0]))

[ 2.3996759e-04  1.5186716e-04 -1.0741219e-04 -8.6460122e-06
  2.2302331e-04  9.4255091e-05  5.3184431e-05  1.8742828e-04
 -2.1787897e-04 -3.1957857e-04 -1.9685674e-04 -2.0849690e-04
  1.1779139e-04  1.5186716e-04  6.0764723e-04  3.0488841e-04
  2.9659504e-04 -2.3486497e-04 -2.3912815e-04 -1.8182107e-04
 -2.3828326e-04 -2.8617471e-04 -9.7164426e-05 -3.2422446e-05
  1.1898611e-04 -4.0961319e-04 -1.0741219e-04  3.0488841e-04
  1.8638392e-03 -2.7468003e-04 -6.5738900e-04 -7.5130764e-04
 -5.4159737e-04 -7.7709946e-04  4.0957029e-04  3.1211090e-04
  6.4047013e-04  1.1044028e-04 -7.2906312e-04 -8.6460122e-06
  2.9659504e-04 -2.7468003e-04  3.7150935e-03 -1.2925579e-03
 -7.6804939e-04  2.1459775e-04 -7.4678764e-04 -9.2790951e-04
 -7.5283402e-04  1.1883125e-03 -3.8307830e-04 -1.1139122e-03
  2.2302331e-04 -2.3486497e-04 -6.5738900e-04 -1.2925579e-03
  3.6607434e-03 -6.0802820e-04  1.2735978e-03  8.3780737e-04
 -3.5398774e-04 -1.1457135e-03 -1.2168454e-03  6.4217886e-05
  3.0574412e-04  9.42550

In [22]:
pd.reset_option('display.max_columns')

Now that all values are proper ndarrays, let's flatten these columns, define a function to flatten a column into multiple new columns containing float64

In [23]:
def flatten_column(df, col):
  print(col)
  result_dict = {}
  num_of_new_cols = max([len(i) for i in df[col]])
  # num_of_new_cols = len(df[col][0])
  num_of_rows = len(df[col])

  for i in range(num_of_new_cols):
    result_col_name = f'{col}_{i}'
    result_dict[result_col_name] = []

  for i in range(num_of_rows):
    for j in range(num_of_new_cols):
      result_col_name = f'{col}_{j}'

      # do padding
      if j >= len(df[col][i]):
        value = 0
      else:
        value = df[col][i][j]
      
      result_dict[result_col_name].append(value)

  return pd.DataFrame(result_dict)

# test the function
df = flatten_column(df_essentia_features_ndarray_columns, 'lowlevel.mfcc.icov')
df

lowlevel.mfcc.icov


,lowlevel.mfcc.icov_0,lowlevel.mfcc.icov_1,lowlevel.mfcc.icov_2,lowlevel.mfcc.icov_3,lowlevel.mfcc.icov_4,lowlevel.mfcc.icov_5,lowlevel.mfcc.icov_6,lowlevel.mfcc.icov_7,lowlevel.mfcc.icov_8,lowlevel.mfcc.icov_9,...,lowlevel.mfcc.icov_159,lowlevel.mfcc.icov_160,lowlevel.mfcc.icov_161,lowlevel.mfcc.icov_162,lowlevel.mfcc.icov_163,lowlevel.mfcc.icov_164,lowlevel.mfcc.icov_165,lowlevel.mfcc.icov_166,lowlevel.mfcc.icov_167,lowlevel.mfcc.icov_168
0,0.000240,0.000152,-0.000107,-0.000009,0.000223,0.000094,0.000053,0.000187,-0.000218,-0.000320,...,-0.001114,0.000306,0.001130,-0.000247,0.000151,-0.001383,-0.000824,-0.000100,-0.004007,0.011772
1,0.000227,0.000240,0.000275,0.000055,-0.000071,-0.000128,-0.000148,-0.000379,0.000045,-0.000398,...,-0.000154,-0.000552,-0.000723,0.000596,-0.000536,0.000177,0.001612,-0.000300,-0.004012,0.009466
2,0.000165,0.000142,0.000083,0.000072,-0.000038,-0.000060,0.000034,-0.000154,0.000055,-0.000038,...,-0.000746,0.000200,-0.000475,0.000306,-0.001661,-0.000076,-0.000839,0.001590,-0.004568,0.013672
3,0.000113,0.000176,0.000148,0.000086,-0.000199,-0.000016,0.000070,-0.000182,-0.000057,0.000139,...,0.001360,-0.001591,-0.000941,0.001233,-0.001503,-0.000423,-0.000782,0.004842,-0.004425,0.013698
4,0.000270,0.000095,0.000131,0.000014,-0.000207,-0.000193,0.000185,-0.000339,-0.000008,0.000014,...,-0.000751,-0.000460,0.000980,-0.000570,-0.002466,-0.000439,-0.000056,-0.000207,-0.004202,0.015842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,0.000181,0.000056,0.000033,0.000056,-0.000136,0.000010,-0.000130,0.000112,-0.000354,-0.000015,...,0.000172,-0.000177,-0.001304,-0.000005,0.000909,-0.000818,-0.001613,0.000835,-0.009341,0.014383
790,0.000065,-0.000018,0.000050,0.000153,-0.000102,-0.000090,0.000020,0.000079,0.000074,-0.000150,...,-0.001655,-0.001934,-0.000285,0.000516,-0.001599,-0.000648,0.001115,-0.003298,-0.008332,0.024900
791,0.000045,-0.000056,-0.000024,-0.000042,-0.000025,-0.000107,0.000003,-0.000068,0.000010,-0.000099,...,-0.000312,-0.000709,-0.001135,0.000525,-0.001735,0.001536,-0.000012,-0.000012,-0.002880,0.011250
792,0.000046,-0.000181,-0.000020,-0.000143,0.000133,-0.000282,0.000167,-0.000076,-0.000067,-0.000081,...,-0.000489,0.003506,-0.001191,0.003284,-0.002186,-0.000435,-0.001631,0.005794,-0.013257,0.023320


Apply the function to flatten all these columns

In [24]:
ndarray_columns = df_essentia_features_ndarray_columns.columns.to_list()
df_ndarray_columns = []

for column in ndarray_columns:
  df_ndarray_column = flatten_column(df_essentia_features_ndarray_columns, column)
  df_ndarray_columns.append(df_ndarray_column)

df_essentia_features_ndarray_columns = pd.concat(df_ndarray_columns, axis=1)
df_essentia_features_ndarray_columns.insert(0, column='song_id', value=song_ids)
df_essentia_features_ndarray_columns

lowlevel.barkbands.dmean
lowlevel.barkbands.dmean2
lowlevel.barkbands.dvar
lowlevel.barkbands.dvar2
lowlevel.barkbands.max
lowlevel.barkbands.mean
lowlevel.barkbands.median
lowlevel.barkbands.min
lowlevel.barkbands.stdev
lowlevel.barkbands.var
lowlevel.erbbands.dmean
lowlevel.erbbands.dmean2
lowlevel.erbbands.dvar
lowlevel.erbbands.dvar2
lowlevel.erbbands.max
lowlevel.erbbands.mean
lowlevel.erbbands.median
lowlevel.erbbands.min
lowlevel.erbbands.stdev
lowlevel.erbbands.var
lowlevel.gfcc.cov
lowlevel.gfcc.icov
lowlevel.gfcc.mean
lowlevel.melbands.dmean
lowlevel.melbands.dmean2
lowlevel.melbands.dvar
lowlevel.melbands.dvar2
lowlevel.melbands.max
lowlevel.melbands.mean
lowlevel.melbands.median
lowlevel.melbands.min
lowlevel.melbands.stdev
lowlevel.melbands.var
lowlevel.melbands128.dmean
lowlevel.melbands128.dmean2
lowlevel.melbands128.dvar
lowlevel.melbands128.dvar2
lowlevel.melbands128.max
lowlevel.melbands128.mean
lowlevel.melbands128.median
lowlevel.melbands128.min
lowlevel.melbands128

,song_id,lowlevel.barkbands.dmean_0,lowlevel.barkbands.dmean_1,lowlevel.barkbands.dmean_2,lowlevel.barkbands.dmean_3,lowlevel.barkbands.dmean_4,lowlevel.barkbands.dmean_5,lowlevel.barkbands.dmean_6,lowlevel.barkbands.dmean_7,lowlevel.barkbands.dmean_8,...,tonal.thpcp_26,tonal.thpcp_27,tonal.thpcp_28,tonal.thpcp_29,tonal.thpcp_30,tonal.thpcp_31,tonal.thpcp_32,tonal.thpcp_33,tonal.thpcp_34,tonal.thpcp_35
0,1,0.000675,0.004961,0.001359,0.001252,0.000917,0.000416,0.001246,0.001042,0.000878,...,0.431222,0.504071,0.259952,0.325301,0.402024,0.279380,0.259622,0.232648,0.244101,0.851880
1,4,0.021252,0.017291,0.003301,0.000898,0.001087,0.000948,0.000958,0.001137,0.000724,...,0.068841,0.130083,0.149334,0.078383,0.057536,0.071149,0.075440,0.050351,0.028523,0.423579
2,5,0.002066,0.017047,0.004393,0.001938,0.001537,0.001425,0.000753,0.000653,0.000347,...,0.062485,0.080485,0.081931,0.070034,0.069636,0.071415,0.080507,0.088939,0.098014,0.743447
3,6,0.001001,0.004160,0.001235,0.000626,0.000872,0.000715,0.000419,0.000303,0.000420,...,0.356605,0.734746,0.499623,0.543463,0.550155,0.140020,0.109984,0.096867,0.093510,0.554280
4,7,0.000508,0.010130,0.006515,0.002177,0.002503,0.001321,0.001311,0.001501,0.001040,...,0.249866,0.615913,0.588765,0.278053,0.244454,0.208635,0.249514,0.483272,0.497619,0.471051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.001496,0.007103,0.002593,0.001464,0.001895,0.001609,0.001196,0.000622,0.000865,...,0.924196,0.959206,0.484637,0.280796,0.235990,0.246565,0.407393,0.462533,0.426415,0.860458
790,996,0.000461,0.006284,0.002041,0.001083,0.001830,0.001626,0.000938,0.000466,0.001002,...,0.362434,0.381794,0.384891,0.602852,0.574937,0.342436,0.281894,0.270360,0.401432,0.889561
791,997,0.000379,0.006677,0.003016,0.001827,0.001161,0.000845,0.000701,0.000542,0.000750,...,0.558074,0.672083,0.445612,0.344640,0.502234,0.482791,0.408797,0.390303,0.378446,0.842631
792,999,0.002049,0.009127,0.003750,0.001479,0.001065,0.000646,0.000369,0.000444,0.000870,...,0.191604,0.166138,0.191558,0.389773,0.433200,0.245289,0.176757,0.187847,0.294250,0.837339


For the string columns, convert these categorical data into numerical data, get the dataframe with only the string columns first

In [25]:
df_essentia_features_string_columns = df_essentia_features[string_columns]
df_essentia_features_string_columns

,tonal.chords_key,tonal.chords_scale,tonal.key_edma.key,tonal.key_edma.scale,tonal.key_krumhansl.key,tonal.key_krumhansl.scale,tonal.key_temperley.key,tonal.key_temperley.scale
0,G,major,G,major,G,major,G,major
1,F#,major,Bb,major,Bb,major,Bb,major
2,A,minor,D,minor,D,minor,D,minor
3,E,minor,C,major,C,major,C,major
4,G,major,G,major,G,major,G,major
...,...,...,...,...,...,...,...,...
789,F,minor,Ab,major,Ab,major,Ab,major
790,C#,minor,C#,minor,C#,minor,C#,minor
791,Eb,major,Ab,major,Ab,major,Ab,major
792,B,minor,B,minor,B,minor,B,minor


Then use cat.codes attribute to convert these categorical columns into numerical columns

In [26]:
for col in df_essentia_features_string_columns.columns:
  df_essentia_features_string_columns[col] = df_essentia_features_string_columns[col].astype('category')
  df_essentia_features_string_columns[col] = df_essentia_features_string_columns[col].cat.codes

df_essentia_features_string_columns.insert(0, column='song_id', value=song_ids)
df_essentia_features_string_columns

/var/folders/ft/gzvmkgdj727g157scc1xw9f80000gn/T/ipykernel_66615/4037814126.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_essentia_features_string_columns[col] = df_essentia_features_string_columns[col].astype('category')
/var/folders/ft/gzvmkgdj727g157scc1xw9f80000gn/T/ipykernel_66615/4037814126.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_essentia_features_string_columns[col] = df_essentia_features_string_columns[col].cat.codes
/var/folders/ft/gzvmkgdj727g157scc1xw9f80000gn/T/ipykernel_

,song_id,tonal.chords_key,tonal.chords_scale,tonal.key_edma.key,tonal.key_edma.scale,tonal.key_krumhansl.key,tonal.key_krumhansl.scale,tonal.key_temperley.key,tonal.key_temperley.scale
0,1,11,0,11,0,11,0,11,0
1,4,10,0,3,0,3,0,3,0
2,5,0,1,6,1,6,1,6,1
3,6,7,1,4,0,4,0,4,0
4,7,11,0,11,0,11,0,11,0
...,...,...,...,...,...,...,...,...,...
789,993,9,1,1,0,1,0,1,0
790,996,5,1,5,1,5,1,5,1
791,997,8,0,1,0,1,0,1,0
792,999,2,1,2,1,2,1,2,1


Combine the ndarray columns, string columns, and the rest of the dataframe together in one flattened dataframe with just numerical data

In [27]:
df_essentia_features_numerical_columns = df_essentia_features.select_dtypes(include=['int64', 'float64'])

df_temp = pd.merge(df_essentia_features_numerical_columns, df_essentia_features_ndarray_columns, how='inner', on='song_id')
df_essentia_features_flattened = pd.merge(df_temp, df_essentia_features_string_columns, how='inner', on='song_id')

df_essentia_features_flattened

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,tonal.thpcp_34,tonal.thpcp_35,tonal.chords_key,tonal.chords_scale,tonal.key_edma.key,tonal.key_edma.scale,tonal.key_krumhansl.key,tonal.key_krumhansl.scale,tonal.key_temperley.key,tonal.key_temperley.scale
0,1,0.970365,2.386482,3.594657,6.740679,14.444414,25.484644,11.411017,10.707484,2.498400,...,0.244101,0.851880,11,0,11,0,11,0,11,0
1,4,0.938222,2.208391,3.674144,5.080004,13.383171,24.118259,14.166083,14.635095,4.408802,...,0.028523,0.423579,10,0,3,0,3,0,3,0
2,5,0.930816,2.393671,3.616534,9.753744,20.003422,26.715527,18.050819,18.184444,2.864332,...,0.098014,0.743447,0,1,6,1,6,1,6,1
3,6,0.954429,1.709188,2.437101,4.900128,8.548371,24.963417,17.043421,18.244125,2.253785,...,0.093510,0.554280,7,1,4,0,4,0,4,0
4,7,0.970629,3.049778,4.716875,10.767557,23.465727,25.671312,10.193353,8.912777,2.406512,...,0.497619,0.471051,11,0,11,0,11,0,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.900841,2.860650,4.524299,7.774405,19.612854,24.813204,11.216174,10.689340,2.157647,...,0.426415,0.860458,9,1,1,0,1,0,1,0
790,996,0.923707,2.780330,4.445892,9.029428,20.219875,25.388409,10.426634,8.341273,2.059642,...,0.401432,0.889561,5,1,5,1,5,1,5,1
791,997,0.913454,2.712144,4.339530,7.626777,20.528904,25.967821,11.849738,11.506009,2.347258,...,0.378446,0.842631,8,0,1,0,1,0,1,0
792,999,0.955402,2.754516,3.943231,5.839927,11.535942,25.688959,11.403626,9.805414,2.655332,...,0.294250,0.837339,2,1,2,1,2,1,2,1


Export the flattened Essentia features dataset

In [28]:
df_essentia_features_flattened.to_csv(get_pmemo_path('processed/features/essentia_features_flattened.csv'))

In [29]:
df_essentia_features_flattened = pd.read_csv(get_pmemo_path('processed/features/essentia_features_flattened.csv'))

## Create separate, more distinct features dataframes extracted by Essentia

Import the features list

In [30]:
from pmemo_essentia_best_features import *

### Get song ids for songs

In [31]:
audio_path = get_pmemo_path('chorus')
song_ids_temp = []

# Iterate through all files in the directory
for filename in os.listdir(audio_path):
    # Check if the path is a file (not a subdirectory)
    if os.path.isfile(os.path.join(audio_path, filename)):
        song_ids_temp.append(int(filename[:-4]))

song_ids_temp.sort()
song_ids = []

# remove all song_ids from 2015 (song_id 2001 onwards)
for song_id in song_ids_temp:
    if song_id <= 2000:
        song_ids.append(song_id)

### Create separate feature dataset for best features for building arousal regressor
According to https://ieeexplore-ieee-org.library.sutd.edu.sg:2443/stamp/stamp.jsp?tp=&arnumber=8001129

In [32]:
df_essentia_best_arousal_features = df_essentia_features_flattened[pmemo_essentia_best_arousal_features_flattened]
df_essentia_best_arousal_features.insert(0, 'song_id', song_ids)
df_essentia_best_arousal_features.to_csv(get_pmemo_path('processed/features/essentia_best_arousal_features.csv'))

Import the best feature dataset .csv for arousal

In [33]:
df_essentia_best_arousal_features = pd.read_csv(get_pmemo_path('processed/features/essentia_best_arousal_features.csv'))

# drop Unnamed:0 column
df_essentia_best_arousal_features = df_essentia_best_arousal_features[df_essentia_best_arousal_features.columns[1:]]

df_essentia_best_arousal_features

,song_id,lowlevel.average_loudness,lowlevel.barkbands_spread.dmean,lowlevel.barkbands_spread.dmean2,lowlevel.barkbands_spread.dvar,lowlevel.barkbands_spread.dvar2,lowlevel.barkbands_spread.max,lowlevel.barkbands_spread.mean,lowlevel.barkbands_spread.median,lowlevel.barkbands_spread.min,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,1,0.970365,9.962052,14.903063,160.617004,331.039124,112.633751,30.026752,25.740023,0.681941,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.630969,0.000000,10.998553,7.814761,18.958033
1,4,0.938222,4.796559,7.069628,64.340286,119.074539,79.542397,8.677779,4.292209,0.278884,...,0.991735,4.793388,25.785124,0.000000,0.661157,6.115703,0.000000,5.289256,0.000000,10.247934
2,5,0.930816,8.162500,12.375443,168.968216,342.220551,117.832550,13.578498,5.963892,0.096577,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.441745,0.220872,0.000000,0.000000,0.000000
3,6,0.954429,8.911726,13.763487,122.484230,282.592804,78.847557,20.610300,16.304569,0.695686,...,0.000000,3.930818,0.000000,0.000000,0.000000,0.000000,0.000000,2.515723,0.000000,0.000000
4,7,0.970629,7.519822,11.455831,102.278740,221.926697,97.466393,23.933802,18.153156,0.721484,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.017544,13.078150,6.379585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.900841,6.240107,9.745099,77.798691,181.858902,117.897911,23.742342,21.268417,0.768844,...,0.000000,0.000000,0.000000,2.113821,0.000000,0.000000,0.000000,0.650406,12.357723,56.829270
790,996,0.923707,8.706299,13.284878,103.574509,212.724503,98.878693,29.687521,27.660221,0.308236,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.521780,1.112141
791,997,0.913454,8.977397,13.752587,112.814796,247.585236,128.250137,39.045204,36.508698,1.025961,...,0.479616,1.598721,0.000000,3.597122,0.000000,1.438849,0.000000,7.354117,2.797762,9.832134
792,999,0.955402,9.373850,14.096522,99.586502,224.760956,120.191139,44.180817,44.341572,3.119685,...,0.000000,0.000000,0.000000,0.000000,0.118765,0.000000,1.543943,0.000000,8.907364,1.306413


### Create separate feature dataset for best features for building valence regressor
According to https://ieeexplore-ieee-org.library.sutd.edu.sg:2443/stamp/stamp.jsp?tp=&arnumber=8001129

In [34]:
df_essentia_best_valence_features = df_essentia_features_flattened[pmemo_essentia_best_valence_features_flattened]
df_essentia_best_valence_features.insert(0, 'song_id', song_ids)
df_essentia_best_valence_features.to_csv(get_pmemo_path('processed/features/essentia_best_valence_features.csv'))

Import the best feature dataset .csv for valence

In [35]:
df_essentia_best_valence_features = pd.read_csv(get_pmemo_path('processed/features/essentia_best_valence_features.csv'))

# drop Unnamed:0 column
df_essentia_best_valence_features = df_essentia_best_valence_features[df_essentia_best_valence_features.columns[1:]]

df_essentia_best_valence_features

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,1,12.878959,19.579144,1312.717041,3060.723389,639.860474,19.044914,5.928576,-1.777708,44.479679,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.630969,0.000000,10.998553,7.814761,18.958033
1,4,47.323818,71.870140,7632.717773,15035.108398,1229.549561,95.002518,40.996933,-1.586174,143.324692,...,0.991735,4.793388,25.785124,0.000000,0.661157,6.115703,0.000000,5.289256,0.000000,10.247934
2,5,100.432968,153.566513,32880.425781,74328.078125,2090.814209,154.612579,60.239502,-1.944383,246.793365,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.441745,0.220872,0.000000,0.000000,0.000000
3,6,23.244177,35.301132,1514.325195,3573.641113,353.348297,47.095280,28.833210,-1.731773,55.418545,...,0.000000,3.930818,0.000000,0.000000,0.000000,0.000000,0.000000,2.515723,0.000000,0.000000
4,7,11.443307,17.701553,695.618958,1591.793823,481.152069,15.404405,7.509385,-1.873933,27.474340,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.017544,13.078150,6.379585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,9.660586,15.619761,918.862549,2431.855225,941.064697,17.451988,7.082492,-1.941114,33.566727,...,0.000000,0.000000,0.000000,2.113821,0.000000,0.000000,0.000000,0.650406,12.357723,56.829270
790,996,19.273998,26.486744,3824.638184,7618.275879,1317.343872,34.581482,5.004123,-1.680010,104.511772,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.521780,1.112141
791,997,8.544683,13.374214,346.121216,893.072449,364.284393,14.138631,6.815628,-1.910969,21.804148,...,0.479616,1.598721,0.000000,3.597122,0.000000,1.438849,0.000000,7.354117,2.797762,9.832134
792,999,9.426803,14.378345,278.683746,670.824463,287.886932,18.854839,8.583271,-1.448540,28.539162,...,0.000000,0.000000,0.000000,0.000000,0.118765,0.000000,1.543943,0.000000,8.907364,1.306413


### Create separate feature dataset for best overall features for detecting both arousal and valence

According to https://ieeexplore-ieee-org.library.sutd.edu.sg:2443/stamp/stamp.jsp?tp=&arnumber=8001129

In [36]:
df_essentia_best_overall_features = df_essentia_features_flattened[pmemo_essentia_best_overall_features_flattened]
df_essentia_best_overall_features.insert(0, 'song_id', song_ids)
df_essentia_best_overall_features.to_csv(get_pmemo_path('processed/features/essentia_best_overall_features.csv'))

Import the best feature dataset .csv overall 

In [37]:
df_essentia_best_overall_features = pd.read_csv(get_pmemo_path('processed/features/essentia_best_overall_features.csv'))

# drop Unnamed:0 column
df_essentia_best_overall_features = df_essentia_best_overall_features[df_essentia_best_overall_features.columns[1:]]

df_essentia_best_overall_features

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,1,12.878959,19.579144,1312.717041,3060.723389,639.860474,19.044914,5.928576,-1.777708,44.479679,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.630969,0.000000,10.998553,7.814761,18.958033
1,4,47.323818,71.870140,7632.717773,15035.108398,1229.549561,95.002518,40.996933,-1.586174,143.324692,...,0.991735,4.793388,25.785124,0.000000,0.661157,6.115703,0.000000,5.289256,0.000000,10.247934
2,5,100.432968,153.566513,32880.425781,74328.078125,2090.814209,154.612579,60.239502,-1.944383,246.793365,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.441745,0.220872,0.000000,0.000000,0.000000
3,6,23.244177,35.301132,1514.325195,3573.641113,353.348297,47.095280,28.833210,-1.731773,55.418545,...,0.000000,3.930818,0.000000,0.000000,0.000000,0.000000,0.000000,2.515723,0.000000,0.000000
4,7,11.443307,17.701553,695.618958,1591.793823,481.152069,15.404405,7.509385,-1.873933,27.474340,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.017544,13.078150,6.379585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,9.660586,15.619761,918.862549,2431.855225,941.064697,17.451988,7.082492,-1.941114,33.566727,...,0.000000,0.000000,0.000000,2.113821,0.000000,0.000000,0.000000,0.650406,12.357723,56.829270
790,996,19.273998,26.486744,3824.638184,7618.275879,1317.343872,34.581482,5.004123,-1.680010,104.511772,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.521780,1.112141
791,997,8.544683,13.374214,346.121216,893.072449,364.284393,14.138631,6.815628,-1.910969,21.804148,...,0.479616,1.598721,0.000000,3.597122,0.000000,1.438849,0.000000,7.354117,2.797762,9.832134
792,999,9.426803,14.378345,278.683746,670.824463,287.886932,18.854839,8.583271,-1.448540,28.539162,...,0.000000,0.000000,0.000000,0.000000,0.118765,0.000000,1.543943,0.000000,8.907364,1.306413


## Integrate Essentia features into openSMILE features

Import openSMILE featuresets .csv

In [38]:
df_pmemo_opensmile_compare2016_features = pd.read_csv(get_pmemo_path('processed/features/opensmile_compare2016_features.csv'))
df_pmemo_opensmile_emobase_features = pd.read_csv(get_pmemo_path('processed/features/opensmile_emobase_features.csv'))
df_pmemo_opensmile_gemaps_features = pd.read_csv(get_pmemo_path('processed/features/opensmile_gemaps_features.csv'))
df_pmemo_opensmile_egemaps_features = pd.read_csv(get_pmemo_path('processed/features/opensmile_egemaps_features.csv'))

# drop Unnamed:0 column
df_pmemo_opensmile_compare2016_features = df_pmemo_opensmile_compare2016_features[df_pmemo_opensmile_compare2016_features.columns[1:]]
df_pmemo_opensmile_emobase_features = df_pmemo_opensmile_emobase_features[df_pmemo_opensmile_emobase_features.columns[1:]]
df_pmemo_opensmile_gemaps_features = df_pmemo_opensmile_gemaps_features[df_pmemo_opensmile_gemaps_features.columns[1:]]
df_pmemo_opensmile_egemaps_features = df_pmemo_opensmile_egemaps_features[df_pmemo_opensmile_egemaps_features.columns[1:]]

In [39]:
df_pmemo_opensmile_compare2016_features

,song_id,audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,audspec_lengthL1norm_sma_iqr2-3,audspec_lengthL1norm_sma_iqr1-3,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,1,7.322202,0.160250,0.0,2.621674,3.417233,4.200210,0.795559,0.782976,1.578536,...,9.387893,0.527035,2.584944,2.586223,-20.000000,0.636703,108.890144,50.417366,107.016620,58.077972
1,4,6.582460,0.703399,0.0,1.656607,2.151616,2.964223,0.495009,0.812608,1.307617,...,7.649520,0.493504,2.592234,2.594137,-19.999989,0.637455,104.337410,59.133785,104.376335,56.402073
2,5,8.124249,0.368032,0.0,1.477557,2.042911,2.738106,0.565354,0.695195,1.260549,...,9.301864,0.578882,2.421328,2.420420,20.000000,0.547071,104.202470,53.521988,107.850940,53.353096
3,6,9.011615,0.281728,0.0,2.313058,3.485445,4.404951,1.172387,0.919506,2.091893,...,9.001396,0.577419,2.309364,2.310359,-20.000000,0.491508,97.956540,49.070080,99.153725,51.412000
4,7,8.349628,0.942354,0.0,4.132348,5.013152,5.750350,0.880803,0.737199,1.618002,...,8.049467,0.513050,2.204685,2.211791,-19.971230,0.553399,94.180350,49.567486,91.844630,46.409393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,8.483475,0.763250,0.0,3.067161,4.405553,5.770013,1.338392,1.364460,2.702852,...,9.415093,0.519761,2.557367,2.557284,20.000000,0.653659,103.939530,59.248730,107.771060,58.497486
790,996,8.911368,0.202281,0.0,4.574404,6.361631,7.147145,1.787228,0.785513,2.572741,...,9.883374,0.546208,2.423050,2.423151,-20.000000,0.616050,112.232630,57.986270,107.318695,52.195194
791,997,8.573480,0.461791,0.0,2.674772,5.131298,6.267901,2.456525,1.136603,3.593128,...,9.722556,0.604539,2.208963,2.208760,20.000000,0.511724,104.249886,53.459106,100.819820,54.904305
792,999,8.656339,0.792811,0.0,4.998912,6.388689,7.008387,1.389777,0.619698,2.009475,...,8.010264,0.556838,2.027305,2.026831,20.000000,0.565515,98.668120,51.425926,96.999370,52.407906


In [40]:
df_pmemo_opensmile_emobase_features

,song_id,pcm_intensity_sma_max,pcm_intensity_sma_min,pcm_intensity_sma_range,pcm_intensity_sma_maxPos,pcm_intensity_sma_minPos,pcm_intensity_sma_amean,pcm_intensity_sma_linregc1,pcm_intensity_sma_linregc2,pcm_intensity_sma_linregerrA,...,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
0,1,0.000069,0.0,0.000069,680.0,0.0,0.000010,1.561768e-09,0.000007,0.000007,...,24.663260,4.969058,1.753136,23.013176,-0.823868,0.0,0.684412,0.823868,0.684412,1.508279
1,4,0.000203,0.0,0.000203,2735.0,0.0,0.000035,9.982415e-09,0.000021,0.000025,...,6.708408,2.596165,0.676692,38.048588,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,5,0.000207,0.0,0.000207,7809.0,0.0,0.000025,9.522875e-10,0.000021,0.000023,...,31.890630,5.647370,0.340074,6.558109,-1.443779,0.0,1.079922,1.443779,1.079922,2.523702
3,6,0.000145,0.0,0.000145,324.0,0.0,0.000025,-4.371458e-09,0.000031,0.000022,...,42.263218,6.504123,-0.035226,4.723821,-2.500313,0.0,2.730563,2.500313,2.730563,5.230876
4,7,0.000140,0.0,0.000140,1182.0,0.0,0.000024,6.233813e-09,0.000015,0.000018,...,16.184336,4.024880,2.228209,31.618954,-0.120087,0.0,0.047111,0.120087,0.047111,0.167197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.000200,0.0,0.000200,4500.0,0.0,0.000025,-1.041277e-09,0.000028,0.000022,...,21.283978,4.614609,1.045086,23.684689,-0.047910,0.0,0.427303,0.047910,0.427303,0.475213
790,996,0.000157,0.0,0.000157,865.0,0.0,0.000028,-3.948720e-10,0.000029,0.000020,...,16.420145,4.055081,0.161662,11.100394,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
791,997,0.000161,0.0,0.000161,589.0,0.0,0.000024,-7.007779e-09,0.000045,0.000017,...,16.416569,4.052447,1.994095,28.915573,-0.340640,0.0,0.133139,0.340640,0.133139,0.473779
792,999,0.000155,0.0,0.000155,2217.0,0.0,0.000032,-8.242266e-09,0.000048,0.000022,...,21.666965,4.656510,0.786846,19.122852,0.000000,0.0,0.006744,0.000000,0.006744,0.006744


In [41]:
df_pmemo_opensmile_gemaps_features

,song_id,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,1,33.337425,0.292724,26.119843,36.004000,40.604490,14.484646,305.86212,450.777680,146.386750,...,-9.104095,18.212526,-0.064266,-0.011526,3.623868,3.161190,0.175842,0.189566,0.130000,0.104119
1,4,28.663845,0.257283,22.888890,28.891180,34.114970,11.226082,181.81364,127.028496,60.697544,...,-23.523148,34.651325,-0.106618,-0.022618,4.177080,0.500715,0.465000,0.899490,1.410667,1.558009
2,5,17.131292,0.444438,12.300488,12.616845,24.207296,11.906808,164.74464,236.056610,100.181330,...,-22.474178,32.335780,-0.098165,-0.014625,4.177577,2.227251,0.301123,0.359941,0.153117,0.170928
3,6,27.172243,0.447355,13.918818,25.472513,43.135548,29.216728,311.85764,277.610320,97.866790,...,-13.287336,22.688347,-0.116574,-0.008948,3.565365,1.734694,0.103922,0.143323,0.489787,0.522769
4,7,27.857573,0.326031,16.902195,30.955670,36.047790,19.145596,331.95724,427.345400,121.956184,...,-6.551864,12.755936,-0.000146,-0.015459,3.375818,2.485330,0.342917,0.373298,0.054167,0.046226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,28.589037,0.445477,16.152197,24.940647,44.368267,28.216070,673.32890,974.933530,230.685240,...,-5.758517,12.847198,-0.031968,-0.011104,2.822581,3.123903,0.203596,0.721575,0.106313,0.100384
790,996,34.963680,0.308940,23.050102,39.976578,43.117737,20.067635,337.40967,449.238000,146.028760,...,-6.366639,16.632452,-0.027622,-0.003730,3.517186,3.542834,0.147345,0.171755,0.125217,0.128034
791,997,23.521826,0.515144,13.664935,16.773577,37.080620,23.415684,354.20798,692.777160,256.592250,...,-4.795950,14.046479,-0.042555,-0.007021,3.032914,2.966540,0.219593,0.315522,0.112945,0.254109
792,999,28.412848,0.529474,14.129504,21.055300,48.873077,34.743572,832.96234,972.408140,152.829000,...,-5.796313,14.950939,-0.058397,-0.006240,2.307101,3.567762,0.128705,0.298521,0.140853,0.374921


In [42]:
df_pmemo_opensmile_egemaps_features

,song_id,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,1,33.337425,0.292724,26.119843,36.004000,40.604490,14.484646,305.86212,450.777680,146.386750,...,-0.064266,-0.011526,3.343537,3.623868,3.161190,0.175842,0.189566,0.130000,0.104119,-14.376077
1,4,28.663845,0.257283,22.888890,28.891180,34.114970,11.226082,181.81364,127.028496,60.697544,...,-0.106618,-0.022618,9.082945,4.177080,0.500715,0.465000,0.899490,1.410667,1.558009,-8.796783
2,5,17.131292,0.444438,12.300488,12.616845,24.207296,11.906808,164.74464,236.056610,100.181330,...,-0.098165,-0.014625,4.480471,4.177577,2.227251,0.301123,0.359941,0.153117,0.170928,-10.281398
3,6,27.172243,0.447355,13.918818,25.472513,43.135548,29.216728,311.85764,277.610320,97.866790,...,-0.116574,-0.008948,3.457023,3.565365,1.734694,0.103922,0.143323,0.489787,0.522769,-10.272443
4,7,27.857573,0.326031,16.902195,30.955670,36.047790,19.145596,331.95724,427.345400,121.956184,...,-0.000146,-0.015459,4.280516,3.375818,2.485330,0.342917,0.373298,0.054167,0.046226,-10.382742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,28.589037,0.445477,16.152197,24.940647,44.368267,28.216070,673.32890,974.933530,230.685240,...,-0.031968,-0.011104,5.368369,2.822581,3.123903,0.203596,0.721575,0.106313,0.100384,-10.396038
790,996,34.963680,0.308940,23.050102,39.976578,43.117737,20.067635,337.40967,449.238000,146.028760,...,-0.027622,-0.003730,5.295775,3.517186,3.542834,0.147345,0.171755,0.125217,0.128034,-9.722353
791,997,23.521826,0.515144,13.664935,16.773577,37.080620,23.415684,354.20798,692.777160,256.592250,...,-0.042555,-0.007021,3.952860,3.032914,2.966540,0.219593,0.315522,0.112945,0.254109,-10.379034
792,999,28.412848,0.529474,14.129504,21.055300,48.873077,34.743572,832.96234,972.408140,152.829000,...,-0.058397,-0.006240,5.495892,2.307101,3.567762,0.128705,0.298521,0.140853,0.374921,-8.888767


Integrate Essentia all features into openSMILE ComParE2016 features

In [43]:
df_pmemo_integrated_essentia_all_opensmile_compare2016 = pd.merge(df_essentia_features_flattened, df_pmemo_opensmile_compare2016_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_pmemo_integrated_essentia_all_opensmile_compare2016.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_pmemo_integrated_essentia_all_opensmile_compare2016.drop(columns=identical_cols, inplace=True)

# drop Unnamed:0 column
df_pmemo_integrated_essentia_all_opensmile_compare2016 = df_pmemo_integrated_essentia_all_opensmile_compare2016[df_pmemo_integrated_essentia_all_opensmile_compare2016.columns[1:]]

df_pmemo_integrated_essentia_all_opensmile_compare2016

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,1,0.970365,2.386482,3.594657,6.740679,14.444414,25.484644,11.411017,10.707484,2.498400,...,9.387893,0.527035,2.584944,2.586223,-20.000000,0.636703,108.890144,50.417366,107.016620,58.077972
1,4,0.938222,2.208391,3.674144,5.080004,13.383171,24.118259,14.166083,14.635095,4.408802,...,7.649520,0.493504,2.592234,2.594137,-19.999989,0.637455,104.337410,59.133785,104.376335,56.402073
2,5,0.930816,2.393671,3.616534,9.753744,20.003422,26.715527,18.050819,18.184444,2.864332,...,9.301864,0.578882,2.421328,2.420420,20.000000,0.547071,104.202470,53.521988,107.850940,53.353096
3,6,0.954429,1.709188,2.437101,4.900128,8.548371,24.963417,17.043421,18.244125,2.253785,...,9.001396,0.577419,2.309364,2.310359,-20.000000,0.491508,97.956540,49.070080,99.153725,51.412000
4,7,0.970629,3.049778,4.716875,10.767557,23.465727,25.671312,10.193353,8.912777,2.406512,...,8.049467,0.513050,2.204685,2.211791,-19.971230,0.553399,94.180350,49.567486,91.844630,46.409393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.900841,2.860650,4.524299,7.774405,19.612854,24.813204,11.216174,10.689340,2.157647,...,9.415093,0.519761,2.557367,2.557284,20.000000,0.653659,103.939530,59.248730,107.771060,58.497486
790,996,0.923707,2.780330,4.445892,9.029428,20.219875,25.388409,10.426634,8.341273,2.059642,...,9.883374,0.546208,2.423050,2.423151,-20.000000,0.616050,112.232630,57.986270,107.318695,52.195194
791,997,0.913454,2.712144,4.339530,7.626777,20.528904,25.967821,11.849738,11.506009,2.347258,...,9.722556,0.604539,2.208963,2.208760,20.000000,0.511724,104.249886,53.459106,100.819820,54.904305
792,999,0.955402,2.754516,3.943231,5.839927,11.535942,25.688959,11.403626,9.805414,2.655332,...,8.010264,0.556838,2.027305,2.026831,20.000000,0.565515,98.668120,51.425926,96.999370,52.407906


In [44]:
df_pmemo_integrated_essentia_all_opensmile_compare2016.to_csv(get_pmemo_path('processed/features/integrated/essentia_all_opensmile_compare2016_features.csv'))

Integrate Essentia all features into openSMILE emobase features

In [45]:
df_pmemo_integrated_essentia_all_opensmile_emobase = pd.merge(df_essentia_features_flattened, df_pmemo_opensmile_emobase_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_pmemo_integrated_essentia_all_opensmile_emobase.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_pmemo_integrated_essentia_all_opensmile_emobase.drop(columns=identical_cols, inplace=True)

# drop Unnamed:0 column
df_pmemo_integrated_essentia_all_opensmile_emobase = df_pmemo_integrated_essentia_all_opensmile_emobase[df_pmemo_integrated_essentia_all_opensmile_emobase.columns[1:]]

df_pmemo_integrated_essentia_all_opensmile_emobase

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
0,1,0.970365,2.386482,3.594657,6.740679,14.444414,25.484644,11.411017,10.707484,2.498400,...,24.663260,4.969058,1.753136,23.013176,-0.823868,0.0,0.684412,0.823868,0.684412,1.508279
1,4,0.938222,2.208391,3.674144,5.080004,13.383171,24.118259,14.166083,14.635095,4.408802,...,6.708408,2.596165,0.676692,38.048588,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,5,0.930816,2.393671,3.616534,9.753744,20.003422,26.715527,18.050819,18.184444,2.864332,...,31.890630,5.647370,0.340074,6.558109,-1.443779,0.0,1.079922,1.443779,1.079922,2.523702
3,6,0.954429,1.709188,2.437101,4.900128,8.548371,24.963417,17.043421,18.244125,2.253785,...,42.263218,6.504123,-0.035226,4.723821,-2.500313,0.0,2.730563,2.500313,2.730563,5.230876
4,7,0.970629,3.049778,4.716875,10.767557,23.465727,25.671312,10.193353,8.912777,2.406512,...,16.184336,4.024880,2.228209,31.618954,-0.120087,0.0,0.047111,0.120087,0.047111,0.167197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.900841,2.860650,4.524299,7.774405,19.612854,24.813204,11.216174,10.689340,2.157647,...,21.283978,4.614609,1.045086,23.684689,-0.047910,0.0,0.427303,0.047910,0.427303,0.475213
790,996,0.923707,2.780330,4.445892,9.029428,20.219875,25.388409,10.426634,8.341273,2.059642,...,16.420145,4.055081,0.161662,11.100394,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
791,997,0.913454,2.712144,4.339530,7.626777,20.528904,25.967821,11.849738,11.506009,2.347258,...,16.416569,4.052447,1.994095,28.915573,-0.340640,0.0,0.133139,0.340640,0.133139,0.473779
792,999,0.955402,2.754516,3.943231,5.839927,11.535942,25.688959,11.403626,9.805414,2.655332,...,21.666965,4.656510,0.786846,19.122852,0.000000,0.0,0.006744,0.000000,0.006744,0.006744


In [46]:
df_pmemo_integrated_essentia_all_opensmile_emobase.to_csv(get_pmemo_path('processed/features/integrated/essentia_all_opensmile_emobase_features.csv'))

Integrate Essentia all features into openSMILE GeMAPS features

In [47]:
df_pmemo_integrated_essentia_all_opensmile_gemaps = pd.merge(df_essentia_features_flattened, df_pmemo_opensmile_gemaps_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_pmemo_integrated_essentia_all_opensmile_gemaps.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_pmemo_integrated_essentia_all_opensmile_gemaps.drop(columns=identical_cols, inplace=True)

# drop Unnamed:0 column
df_pmemo_integrated_essentia_all_opensmile_gemaps = df_pmemo_integrated_essentia_all_opensmile_gemaps[df_pmemo_integrated_essentia_all_opensmile_gemaps.columns[1:]]

df_pmemo_integrated_essentia_all_opensmile_gemaps

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,1,0.970365,2.386482,3.594657,6.740679,14.444414,25.484644,11.411017,10.707484,2.498400,...,-9.104095,18.212526,-0.064266,-0.011526,3.623868,3.161190,0.175842,0.189566,0.130000,0.104119
1,4,0.938222,2.208391,3.674144,5.080004,13.383171,24.118259,14.166083,14.635095,4.408802,...,-23.523148,34.651325,-0.106618,-0.022618,4.177080,0.500715,0.465000,0.899490,1.410667,1.558009
2,5,0.930816,2.393671,3.616534,9.753744,20.003422,26.715527,18.050819,18.184444,2.864332,...,-22.474178,32.335780,-0.098165,-0.014625,4.177577,2.227251,0.301123,0.359941,0.153117,0.170928
3,6,0.954429,1.709188,2.437101,4.900128,8.548371,24.963417,17.043421,18.244125,2.253785,...,-13.287336,22.688347,-0.116574,-0.008948,3.565365,1.734694,0.103922,0.143323,0.489787,0.522769
4,7,0.970629,3.049778,4.716875,10.767557,23.465727,25.671312,10.193353,8.912777,2.406512,...,-6.551864,12.755936,-0.000146,-0.015459,3.375818,2.485330,0.342917,0.373298,0.054167,0.046226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.900841,2.860650,4.524299,7.774405,19.612854,24.813204,11.216174,10.689340,2.157647,...,-5.758517,12.847198,-0.031968,-0.011104,2.822581,3.123903,0.203596,0.721575,0.106313,0.100384
790,996,0.923707,2.780330,4.445892,9.029428,20.219875,25.388409,10.426634,8.341273,2.059642,...,-6.366639,16.632452,-0.027622,-0.003730,3.517186,3.542834,0.147345,0.171755,0.125217,0.128034
791,997,0.913454,2.712144,4.339530,7.626777,20.528904,25.967821,11.849738,11.506009,2.347258,...,-4.795950,14.046479,-0.042555,-0.007021,3.032914,2.966540,0.219593,0.315522,0.112945,0.254109
792,999,0.955402,2.754516,3.943231,5.839927,11.535942,25.688959,11.403626,9.805414,2.655332,...,-5.796313,14.950939,-0.058397,-0.006240,2.307101,3.567762,0.128705,0.298521,0.140853,0.374921


In [48]:
df_pmemo_integrated_essentia_all_opensmile_gemaps.to_csv(get_pmemo_path('processed/features/integrated/essentia_all_opensmile_gemaps_features.csv'))

Integrate Essentia all features into openSMILE eGeMAPS features

In [49]:
df_pmemo_integrated_essentia_all_opensmile_egemaps = pd.merge(df_essentia_features_flattened, df_pmemo_opensmile_egemaps_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_pmemo_integrated_essentia_all_opensmile_egemaps.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_pmemo_integrated_essentia_all_opensmile_egemaps.drop(columns=identical_cols, inplace=True)

# drop Unnamed:0 column
df_pmemo_integrated_essentia_all_opensmile_egemaps = df_pmemo_integrated_essentia_all_opensmile_egemaps[df_pmemo_integrated_essentia_all_opensmile_egemaps.columns[1:]]

df_pmemo_integrated_essentia_all_opensmile_egemaps

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,1,0.970365,2.386482,3.594657,6.740679,14.444414,25.484644,11.411017,10.707484,2.498400,...,-0.064266,-0.011526,3.343537,3.623868,3.161190,0.175842,0.189566,0.130000,0.104119,-14.376077
1,4,0.938222,2.208391,3.674144,5.080004,13.383171,24.118259,14.166083,14.635095,4.408802,...,-0.106618,-0.022618,9.082945,4.177080,0.500715,0.465000,0.899490,1.410667,1.558009,-8.796783
2,5,0.930816,2.393671,3.616534,9.753744,20.003422,26.715527,18.050819,18.184444,2.864332,...,-0.098165,-0.014625,4.480471,4.177577,2.227251,0.301123,0.359941,0.153117,0.170928,-10.281398
3,6,0.954429,1.709188,2.437101,4.900128,8.548371,24.963417,17.043421,18.244125,2.253785,...,-0.116574,-0.008948,3.457023,3.565365,1.734694,0.103922,0.143323,0.489787,0.522769,-10.272443
4,7,0.970629,3.049778,4.716875,10.767557,23.465727,25.671312,10.193353,8.912777,2.406512,...,-0.000146,-0.015459,4.280516,3.375818,2.485330,0.342917,0.373298,0.054167,0.046226,-10.382742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.900841,2.860650,4.524299,7.774405,19.612854,24.813204,11.216174,10.689340,2.157647,...,-0.031968,-0.011104,5.368369,2.822581,3.123903,0.203596,0.721575,0.106313,0.100384,-10.396038
790,996,0.923707,2.780330,4.445892,9.029428,20.219875,25.388409,10.426634,8.341273,2.059642,...,-0.027622,-0.003730,5.295775,3.517186,3.542834,0.147345,0.171755,0.125217,0.128034,-9.722353
791,997,0.913454,2.712144,4.339530,7.626777,20.528904,25.967821,11.849738,11.506009,2.347258,...,-0.042555,-0.007021,3.952860,3.032914,2.966540,0.219593,0.315522,0.112945,0.254109,-10.379034
792,999,0.955402,2.754516,3.943231,5.839927,11.535942,25.688959,11.403626,9.805414,2.655332,...,-0.058397,-0.006240,5.495892,2.307101,3.567762,0.128705,0.298521,0.140853,0.374921,-8.888767


In [50]:
df_pmemo_integrated_essentia_all_opensmile_egemaps.to_csv(get_pmemo_path('processed/features/integrated/essentia_all_opensmile_egemaps_features.csv'))

Integrate Essentia best overall features into openSMILE ComParE2016 features

In [51]:
df_pmemo_integrated_essentia_best_overall_opensmile_compare2016 = pd.merge(df_essentia_best_overall_features, df_pmemo_opensmile_compare2016_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_pmemo_integrated_essentia_best_overall_opensmile_compare2016.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_pmemo_integrated_essentia_best_overall_opensmile_compare2016.drop(columns=identical_cols, inplace=True)

df_pmemo_integrated_essentia_best_overall_opensmile_compare2016

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,1,12.878959,19.579144,1312.717041,3060.723389,639.860474,19.044914,5.928576,-1.777708,44.479679,...,9.387893,0.527035,2.584944,2.586223,-20.000000,0.636703,108.890144,50.417366,107.016620,58.077972
1,4,47.323818,71.870140,7632.717773,15035.108398,1229.549561,95.002518,40.996933,-1.586174,143.324692,...,7.649520,0.493504,2.592234,2.594137,-19.999989,0.637455,104.337410,59.133785,104.376335,56.402073
2,5,100.432968,153.566513,32880.425781,74328.078125,2090.814209,154.612579,60.239502,-1.944383,246.793365,...,9.301864,0.578882,2.421328,2.420420,20.000000,0.547071,104.202470,53.521988,107.850940,53.353096
3,6,23.244177,35.301132,1514.325195,3573.641113,353.348297,47.095280,28.833210,-1.731773,55.418545,...,9.001396,0.577419,2.309364,2.310359,-20.000000,0.491508,97.956540,49.070080,99.153725,51.412000
4,7,11.443307,17.701553,695.618958,1591.793823,481.152069,15.404405,7.509385,-1.873933,27.474340,...,8.049467,0.513050,2.204685,2.211791,-19.971230,0.553399,94.180350,49.567486,91.844630,46.409393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,9.660586,15.619761,918.862549,2431.855225,941.064697,17.451988,7.082492,-1.941114,33.566727,...,9.415093,0.519761,2.557367,2.557284,20.000000,0.653659,103.939530,59.248730,107.771060,58.497486
790,996,19.273998,26.486744,3824.638184,7618.275879,1317.343872,34.581482,5.004123,-1.680010,104.511772,...,9.883374,0.546208,2.423050,2.423151,-20.000000,0.616050,112.232630,57.986270,107.318695,52.195194
791,997,8.544683,13.374214,346.121216,893.072449,364.284393,14.138631,6.815628,-1.910969,21.804148,...,9.722556,0.604539,2.208963,2.208760,20.000000,0.511724,104.249886,53.459106,100.819820,54.904305
792,999,9.426803,14.378345,278.683746,670.824463,287.886932,18.854839,8.583271,-1.448540,28.539162,...,8.010264,0.556838,2.027305,2.026831,20.000000,0.565515,98.668120,51.425926,96.999370,52.407906


In [52]:
df_pmemo_integrated_essentia_best_overall_opensmile_compare2016.to_csv(get_pmemo_path('processed/features/integrated/essentia_best_overall_opensmile_compare2016_features.csv'))

Integrate Essentia best overall features into openSMILE emobase features

In [53]:
df_pmemo_integrated_essentia_best_overall_opensmile_emobase = pd.merge(df_essentia_best_overall_features, df_pmemo_opensmile_emobase_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_pmemo_integrated_essentia_best_overall_opensmile_emobase.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_pmemo_integrated_essentia_best_overall_opensmile_emobase.drop(columns=identical_cols, inplace=True)

df_pmemo_integrated_essentia_best_overall_opensmile_emobase

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
0,1,12.878959,19.579144,1312.717041,3060.723389,639.860474,19.044914,5.928576,-1.777708,44.479679,...,24.663260,4.969058,1.753136,23.013176,-0.823868,0.0,0.684412,0.823868,0.684412,1.508279
1,4,47.323818,71.870140,7632.717773,15035.108398,1229.549561,95.002518,40.996933,-1.586174,143.324692,...,6.708408,2.596165,0.676692,38.048588,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,5,100.432968,153.566513,32880.425781,74328.078125,2090.814209,154.612579,60.239502,-1.944383,246.793365,...,31.890630,5.647370,0.340074,6.558109,-1.443779,0.0,1.079922,1.443779,1.079922,2.523702
3,6,23.244177,35.301132,1514.325195,3573.641113,353.348297,47.095280,28.833210,-1.731773,55.418545,...,42.263218,6.504123,-0.035226,4.723821,-2.500313,0.0,2.730563,2.500313,2.730563,5.230876
4,7,11.443307,17.701553,695.618958,1591.793823,481.152069,15.404405,7.509385,-1.873933,27.474340,...,16.184336,4.024880,2.228209,31.618954,-0.120087,0.0,0.047111,0.120087,0.047111,0.167197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,9.660586,15.619761,918.862549,2431.855225,941.064697,17.451988,7.082492,-1.941114,33.566727,...,21.283978,4.614609,1.045086,23.684689,-0.047910,0.0,0.427303,0.047910,0.427303,0.475213
790,996,19.273998,26.486744,3824.638184,7618.275879,1317.343872,34.581482,5.004123,-1.680010,104.511772,...,16.420145,4.055081,0.161662,11.100394,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
791,997,8.544683,13.374214,346.121216,893.072449,364.284393,14.138631,6.815628,-1.910969,21.804148,...,16.416569,4.052447,1.994095,28.915573,-0.340640,0.0,0.133139,0.340640,0.133139,0.473779
792,999,9.426803,14.378345,278.683746,670.824463,287.886932,18.854839,8.583271,-1.448540,28.539162,...,21.666965,4.656510,0.786846,19.122852,0.000000,0.0,0.006744,0.000000,0.006744,0.006744


In [54]:
df_pmemo_integrated_essentia_best_overall_opensmile_emobase.to_csv(get_pmemo_path('processed/features/integrated/essentia_best_overall_opensmile_emobase_features.csv'))

Integrate Essentia best overall features into openSMILE GeMAPS features

In [55]:
df_pmemo_integrated_essentia_best_overall_opensmile_gemaps = pd.merge(df_essentia_best_overall_features, df_pmemo_opensmile_gemaps_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_pmemo_integrated_essentia_best_overall_opensmile_gemaps.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_pmemo_integrated_essentia_best_overall_opensmile_gemaps.drop(columns=identical_cols, inplace=True)

df_pmemo_integrated_essentia_best_overall_opensmile_gemaps

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,1,12.878959,19.579144,1312.717041,3060.723389,639.860474,19.044914,5.928576,-1.777708,44.479679,...,-9.104095,18.212526,-0.064266,-0.011526,3.623868,3.161190,0.175842,0.189566,0.130000,0.104119
1,4,47.323818,71.870140,7632.717773,15035.108398,1229.549561,95.002518,40.996933,-1.586174,143.324692,...,-23.523148,34.651325,-0.106618,-0.022618,4.177080,0.500715,0.465000,0.899490,1.410667,1.558009
2,5,100.432968,153.566513,32880.425781,74328.078125,2090.814209,154.612579,60.239502,-1.944383,246.793365,...,-22.474178,32.335780,-0.098165,-0.014625,4.177577,2.227251,0.301123,0.359941,0.153117,0.170928
3,6,23.244177,35.301132,1514.325195,3573.641113,353.348297,47.095280,28.833210,-1.731773,55.418545,...,-13.287336,22.688347,-0.116574,-0.008948,3.565365,1.734694,0.103922,0.143323,0.489787,0.522769
4,7,11.443307,17.701553,695.618958,1591.793823,481.152069,15.404405,7.509385,-1.873933,27.474340,...,-6.551864,12.755936,-0.000146,-0.015459,3.375818,2.485330,0.342917,0.373298,0.054167,0.046226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,9.660586,15.619761,918.862549,2431.855225,941.064697,17.451988,7.082492,-1.941114,33.566727,...,-5.758517,12.847198,-0.031968,-0.011104,2.822581,3.123903,0.203596,0.721575,0.106313,0.100384
790,996,19.273998,26.486744,3824.638184,7618.275879,1317.343872,34.581482,5.004123,-1.680010,104.511772,...,-6.366639,16.632452,-0.027622,-0.003730,3.517186,3.542834,0.147345,0.171755,0.125217,0.128034
791,997,8.544683,13.374214,346.121216,893.072449,364.284393,14.138631,6.815628,-1.910969,21.804148,...,-4.795950,14.046479,-0.042555,-0.007021,3.032914,2.966540,0.219593,0.315522,0.112945,0.254109
792,999,9.426803,14.378345,278.683746,670.824463,287.886932,18.854839,8.583271,-1.448540,28.539162,...,-5.796313,14.950939,-0.058397,-0.006240,2.307101,3.567762,0.128705,0.298521,0.140853,0.374921


In [56]:
df_pmemo_integrated_essentia_best_overall_opensmile_gemaps.to_csv(get_pmemo_path('processed/features/integrated/essentia_best_overall_opensmile_gemaps_features.csv'))

Integrate Essentia best overall features into openSMILE eGeMAPS features

In [57]:
df_pmemo_integrated_essentia_best_overall_opensmile_egemaps = pd.merge(df_essentia_best_overall_features, df_pmemo_opensmile_egemaps_features, on='song_id', how='inner')

# Identify identical columns for dropping
identical_cols = [col for col in df_pmemo_integrated_essentia_best_overall_opensmile_egemaps.columns if '_x' in col or '_y' in col]

# Drop identical columns
df_pmemo_integrated_essentia_best_overall_opensmile_egemaps.drop(columns=identical_cols, inplace=True)

df_pmemo_integrated_essentia_best_overall_opensmile_egemaps

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,1,12.878959,19.579144,1312.717041,3060.723389,639.860474,19.044914,5.928576,-1.777708,44.479679,...,-0.064266,-0.011526,3.343537,3.623868,3.161190,0.175842,0.189566,0.130000,0.104119,-14.376077
1,4,47.323818,71.870140,7632.717773,15035.108398,1229.549561,95.002518,40.996933,-1.586174,143.324692,...,-0.106618,-0.022618,9.082945,4.177080,0.500715,0.465000,0.899490,1.410667,1.558009,-8.796783
2,5,100.432968,153.566513,32880.425781,74328.078125,2090.814209,154.612579,60.239502,-1.944383,246.793365,...,-0.098165,-0.014625,4.480471,4.177577,2.227251,0.301123,0.359941,0.153117,0.170928,-10.281398
3,6,23.244177,35.301132,1514.325195,3573.641113,353.348297,47.095280,28.833210,-1.731773,55.418545,...,-0.116574,-0.008948,3.457023,3.565365,1.734694,0.103922,0.143323,0.489787,0.522769,-10.272443
4,7,11.443307,17.701553,695.618958,1591.793823,481.152069,15.404405,7.509385,-1.873933,27.474340,...,-0.000146,-0.015459,4.280516,3.375818,2.485330,0.342917,0.373298,0.054167,0.046226,-10.382742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,9.660586,15.619761,918.862549,2431.855225,941.064697,17.451988,7.082492,-1.941114,33.566727,...,-0.031968,-0.011104,5.368369,2.822581,3.123903,0.203596,0.721575,0.106313,0.100384,-10.396038
790,996,19.273998,26.486744,3824.638184,7618.275879,1317.343872,34.581482,5.004123,-1.680010,104.511772,...,-0.027622,-0.003730,5.295775,3.517186,3.542834,0.147345,0.171755,0.125217,0.128034,-9.722353
791,997,8.544683,13.374214,346.121216,893.072449,364.284393,14.138631,6.815628,-1.910969,21.804148,...,-0.042555,-0.007021,3.952860,3.032914,2.966540,0.219593,0.315522,0.112945,0.254109,-10.379034
792,999,9.426803,14.378345,278.683746,670.824463,287.886932,18.854839,8.583271,-1.448540,28.539162,...,-0.058397,-0.006240,5.495892,2.307101,3.567762,0.128705,0.298521,0.140853,0.374921,-8.888767


In [58]:
df_pmemo_integrated_essentia_best_overall_opensmile_egemaps.to_csv(get_pmemo_path('processed/features/integrated/essentia_best_overall_opensmile_egemaps_features.csv'))

## Data Scaling

Import relevant libraries

In [59]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

### Data Standardisation

In [60]:
scaler = StandardScaler()

Essentia All + openSMILE ComParE2016

In [61]:
# Fit and transform the selected columns
df_pmemo_integrated_essentia_all_opensmile_compare2016_standardised = pd.DataFrame(scaler.fit_transform(df_pmemo_integrated_essentia_all_opensmile_compare2016), columns=df_pmemo_integrated_essentia_all_opensmile_compare2016.columns)

df_pmemo_integrated_essentia_all_opensmile_compare2016_standardised = df_pmemo_integrated_essentia_all_opensmile_compare2016_standardised.drop('song_id', axis=1)
df_pmemo_integrated_essentia_all_opensmile_compare2016_standardised.insert(0, column='song_id', value=song_ids)

df_pmemo_integrated_essentia_all_opensmile_compare2016_standardised.to_csv(get_pmemo_path('processed/features/integrated/standardised_essentia_all_opensmile_compare2016_features.csv'))

df_pmemo_integrated_essentia_all_opensmile_compare2016_standardised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,1,0.422535,-0.548050,-0.749892,-0.169271,-0.448214,0.407109,-0.101142,-0.118533,-0.556180,...,0.433505,-0.561992,0.843684,0.848693,-1.046653,1.049362,0.419328,-0.405973,0.411406,0.848458
1,4,0.117155,-0.920705,-0.647443,-0.918700,-0.658138,-0.407319,1.002655,1.244151,2.857390,...,-0.587471,-1.137527,0.867144,0.874174,-1.046653,1.061666,0.006723,0.818896,0.158048,0.591951
2,5,0.046795,-0.533008,-0.721695,1.190463,0.651411,1.140771,2.559047,2.475598,0.097679,...,0.382979,0.327910,0.317124,0.314843,0.955835,-0.416433,-0.005506,0.030302,0.491466,0.125287
3,6,0.271131,-1.965287,-2.241840,-0.999874,-1.614508,0.096434,2.155440,2.496304,-0.993267,...,0.206509,0.302809,-0.043210,-0.039529,-1.046653,-1.325080,-0.571561,-0.595299,-0.343105,-0.171810
4,7,0.425040,0.839894,0.696510,1.647976,1.336288,0.518372,-0.588991,-0.741207,-0.720369,...,-0.352575,-0.802031,-0.380093,-0.356896,-1.045213,-0.312954,-0.913789,-0.525402,-1.044475,-0.937489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,-0.237989,0.444144,0.448303,0.297228,0.574153,0.006900,-0.179205,-0.124829,-1.165049,...,0.449480,-0.686836,0.754932,0.755514,0.955835,1.326664,-0.029336,0.835049,0.483801,0.912667
790,996,-0.020750,0.276075,0.347246,0.863594,0.694228,0.349748,-0.495529,-0.939490,-1.340168,...,0.724510,-0.232904,0.322665,0.323636,-1.046653,0.711623,0.722250,0.657643,0.440393,-0.051937
791,997,-0.118164,0.133395,0.210158,0.230607,0.755357,0.695105,0.074628,0.158515,-0.826245,...,0.630059,0.768304,-0.366326,-0.366658,0.955835,-0.994472,-0.001209,0.021466,-0.183229,0.362709
792,999,0.280373,0.222058,-0.300623,-0.575762,-1.023538,0.528890,-0.104104,-0.431506,-0.275769,...,-0.375600,-0.050453,-0.950951,-0.952429,0.955835,-0.114814,-0.507072,-0.264246,-0.549834,-0.019380


Essentia All + openSMILE emobase

In [62]:
# Fit and transform the selected columns
df_pmemo_integrated_essentia_all_opensmile_emobase_standardised = pd.DataFrame(scaler.fit_transform(df_pmemo_integrated_essentia_all_opensmile_emobase), columns=df_pmemo_integrated_essentia_all_opensmile_emobase.columns)

df_pmemo_integrated_essentia_all_opensmile_emobase_standardised = df_pmemo_integrated_essentia_all_opensmile_emobase_standardised.drop('song_id', axis=1)
df_pmemo_integrated_essentia_all_opensmile_emobase_standardised.insert(0, column='song_id', value=song_ids)

df_pmemo_integrated_essentia_all_opensmile_emobase_standardised.to_csv(get_pmemo_path('processed/features/integrated/standardised_essentia_all_opensmile_emobase_features.csv'))

df_pmemo_integrated_essentia_all_opensmile_emobase_standardised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
0,1,0.422535,-0.548050,-0.749892,-0.169271,-0.448214,0.407109,-0.101142,-0.118533,-0.556180,...,0.617745,0.712242,-0.243054,-0.374326,-0.498373,0.067263,0.357042,0.501149,0.354853,0.431765
1,4,0.117155,-0.920705,-0.647443,-0.918700,-0.658138,-0.407319,1.002655,1.244151,2.857390,...,-1.234588,-1.434384,-0.683549,-0.056562,0.708067,0.067263,-0.670308,-0.709353,-0.669416,-0.694541
2,5,0.046795,-0.533008,-0.721695,1.190463,0.651411,1.140771,2.559047,2.475598,0.097679,...,1.363365,1.325873,-0.821297,-0.722094,-1.406147,0.067263,0.950732,1.411980,0.946761,1.190031
3,6,0.271131,-1.965287,-2.241840,-0.999874,-1.614508,0.096434,2.155440,2.496304,-0.993267,...,2.433465,2.100930,-0.974874,-0.760861,-2.953295,0.067263,3.428463,2.964336,3.417057,3.211612
4,7,0.425040,0.839894,0.696510,1.647976,1.336288,0.518372,-0.588991,-0.741207,-0.720369,...,-0.256993,-0.141904,-0.048648,-0.192448,0.532217,0.067263,-0.599592,-0.532911,-0.598911,-0.569687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,-0.237989,0.444144,0.448303,0.297228,0.574153,0.006900,-0.179205,-0.124829,-1.165049,...,0.269117,0.391591,-0.532797,-0.360134,0.637909,0.067263,-0.028897,-0.638959,-0.029929,-0.339677
790,996,-0.020750,0.276075,0.347246,0.863594,0.694228,0.349748,-0.495529,-0.939490,-1.340168,...,-0.232666,-0.114583,-0.894306,-0.626096,0.708067,0.067263,-0.670308,-0.709353,-0.669416,-0.694541
791,997,-0.118164,0.133395,0.210158,0.230607,0.755357,0.695105,0.074628,0.158515,-0.826245,...,-0.233035,-0.116966,-0.144450,-0.249582,0.209247,0.067263,-0.470457,-0.208853,-0.470164,-0.340747
792,999,0.280373,0.222058,-0.300623,-0.575762,-1.023538,0.528890,-0.104104,-0.431506,-0.275769,...,0.308629,0.429497,-0.638472,-0.456546,0.708067,0.067263,-0.660185,-0.709353,-0.659322,-0.689505


Essentia All + openSMILE GeMAPS

In [63]:
# Fit and transform the selected columns
df_pmemo_integrated_essentia_all_opensmile_gemaps_standardised = pd.DataFrame(scaler.fit_transform(df_pmemo_integrated_essentia_all_opensmile_gemaps), columns=df_pmemo_integrated_essentia_all_opensmile_gemaps.columns)

df_pmemo_integrated_essentia_all_opensmile_gemaps_standardised = df_pmemo_integrated_essentia_all_opensmile_gemaps_standardised.drop('song_id', axis=1)
df_pmemo_integrated_essentia_all_opensmile_gemaps_standardised.insert(0, column='song_id', value=song_ids)

df_pmemo_integrated_essentia_all_opensmile_gemaps_standardised.to_csv(get_pmemo_path('processed/features/integrated/standardised_essentia_all_opensmile_gemaps_features.csv'))

df_pmemo_integrated_essentia_all_opensmile_gemaps_standardised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,1,0.422535,-0.548050,-0.749892,-0.169271,-0.448214,0.407109,-0.101142,-0.118533,-0.556180,...,0.014368,0.110506,-0.359317,0.242259,0.626886,0.384981,-0.331465,-0.379390,0.257481,-0.073466
1,4,0.117155,-0.920705,-0.647443,-0.918700,-0.658138,-0.407319,1.002655,1.244151,2.857390,...,-2.567809,2.516917,-1.563992,-1.753440,1.245798,-2.045060,0.047275,0.338540,13.752535,12.618872
2,5,0.046795,-0.533008,-0.721695,1.190463,0.651411,1.140771,2.559047,2.475598,0.097679,...,-2.379958,2.177954,-1.323570,-0.315244,1.246355,-0.468066,-0.167371,-0.207094,0.501076,0.509763
3,6,0.271131,-1.965287,-2.241840,-0.999874,-1.614508,0.096434,2.155440,2.496304,-0.993267,...,-0.734771,0.765704,-1.847202,0.706169,0.561435,-0.917961,-0.425666,-0.426155,4.048747,3.581308
4,7,0.425040,0.839894,0.696510,1.647976,1.336288,0.518372,-0.588991,-0.741207,-0.720369,...,0.471424,-0.688262,1.464579,-0.465351,0.349378,-0.232340,-0.112629,-0.193586,-0.541614,-0.578875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,-0.237989,0.444144,0.448303,0.297228,0.574153,0.006900,-0.179205,-0.124829,-1.165049,...,0.613497,-0.674903,0.559402,0.318206,-0.269563,0.350924,-0.295113,0.158619,0.007874,-0.106073
790,996,-0.020750,0.276075,0.347246,0.863594,0.694228,0.349748,-0.495529,-0.939490,-1.340168,...,0.504594,-0.120795,0.683018,1.644929,0.507535,0.733571,-0.368790,-0.397402,0.207085,0.135304
791,997,-0.118164,0.133395,0.210158,0.230607,0.755357,0.695105,0.074628,0.158515,-0.826245,...,0.785874,-0.499345,0.258260,1.052879,-0.034250,0.207191,-0.274159,-0.252014,0.077766,1.235931
792,999,0.280373,0.222058,-0.300623,-0.575762,-1.023538,0.528890,-0.104104,-0.431506,-0.275769,...,0.606728,-0.366945,-0.192367,1.193343,-0.846263,0.756339,-0.393204,-0.269206,0.371842,2.290611


Essentia All + openSMILE eGeMAPS

In [64]:
# Fit and transform the selected columns
df_pmemo_integrated_essentia_all_opensmile_egemaps_standardised = pd.DataFrame(scaler.fit_transform(df_pmemo_integrated_essentia_all_opensmile_egemaps), columns=df_pmemo_integrated_essentia_all_opensmile_egemaps.columns)

df_pmemo_integrated_essentia_all_opensmile_egemaps_standardised = df_pmemo_integrated_essentia_all_opensmile_egemaps_standardised.drop('song_id', axis=1)
df_pmemo_integrated_essentia_all_opensmile_egemaps_standardised.insert(0, column='song_id', value=song_ids)

df_pmemo_integrated_essentia_all_opensmile_egemaps_standardised.to_csv(get_pmemo_path('processed/features/integrated/standardised_essentia_all_opensmile_egemaps_features.csv'))

df_pmemo_integrated_essentia_all_opensmile_egemaps_standardised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,1,0.422535,-0.548050,-0.749892,-0.169271,-0.448214,0.407109,-0.101142,-0.118533,-0.556180,...,-0.359317,0.242259,-1.044193,0.626886,0.384981,-0.331465,-0.379390,0.257481,-0.073466,-1.152246
1,4,0.117155,-0.920705,-0.647443,-0.918700,-0.658138,-0.407319,1.002655,1.244151,2.857390,...,-1.563992,-1.753440,2.568822,1.245798,-2.045060,0.047275,0.338540,13.752535,12.618872,1.003751
2,5,0.046795,-0.533008,-0.721695,1.190463,0.651411,1.140771,2.559047,2.475598,0.097679,...,-1.323570,-0.315244,-0.328482,1.246355,-0.468066,-0.167371,-0.207094,0.501076,0.509763,0.430054
3,6,0.271131,-1.965287,-2.241840,-0.999874,-1.614508,0.096434,2.155440,2.496304,-0.993267,...,-1.847202,0.706169,-0.972752,0.561435,-0.917961,-0.425666,-0.426155,4.048747,3.581308,0.433514
4,7,0.425040,0.839894,0.696510,1.647976,1.336288,0.518372,-0.588991,-0.741207,-0.720369,...,1.464579,-0.465351,-0.454355,0.349378,-0.232340,-0.112629,-0.193586,-0.541614,-0.578875,0.390891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,-0.237989,0.444144,0.448303,0.297228,0.574153,0.006900,-0.179205,-0.124829,-1.165049,...,0.559402,0.318206,0.230459,-0.269563,0.350924,-0.295113,0.158619,0.007874,-0.106073,0.385753
790,996,-0.020750,0.276075,0.347246,0.863594,0.694228,0.349748,-0.495529,-0.939490,-1.340168,...,0.683018,1.644929,0.184761,0.507535,0.733571,-0.368790,-0.397402,0.207085,0.135304,0.646084
791,997,-0.118164,0.133395,0.210158,0.230607,0.755357,0.695105,0.074628,0.158515,-0.826245,...,0.258260,1.052879,-0.660618,-0.034250,0.207191,-0.274159,-0.252014,0.077766,1.235931,0.392324
792,999,0.280373,0.222058,-0.300623,-0.575762,-1.023538,0.528890,-0.104104,-0.431506,-0.275769,...,-0.192367,1.193343,0.310736,-0.846263,0.756339,-0.393204,-0.269206,0.371842,2.290611,0.968205


Essentia Best Overall + openSMILE ComParE2016

In [65]:
# Fit and transform the selected columns
df_pmemo_integrated_essentia_best_overall_opensmile_compare2016_standardised = pd.DataFrame(scaler.fit_transform(df_pmemo_integrated_essentia_best_overall_opensmile_compare2016), columns=df_pmemo_integrated_essentia_best_overall_opensmile_compare2016.columns)

df_pmemo_integrated_essentia_best_overall_opensmile_compare2016_standardised = df_pmemo_integrated_essentia_best_overall_opensmile_compare2016_standardised.drop('song_id', axis=1)
df_pmemo_integrated_essentia_best_overall_opensmile_compare2016_standardised.insert(0, column='song_id', value=song_ids)

df_pmemo_integrated_essentia_best_overall_opensmile_compare2016_standardised.to_csv(get_pmemo_path('processed/features/integrated/standardised_essentia_best_overall_opensmile_compare2016_features.csv'))

df_pmemo_integrated_essentia_best_overall_opensmile_compare2016_standardised

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,1,-0.212798,-0.221286,-0.207779,-0.192209,0.097903,-0.282657,-0.373510,-0.792719,-0.098023,...,0.433505,-0.561992,0.843684,0.848693,-1.046653,1.049362,0.419328,-0.405973,0.411406,0.848458
1,4,1.398278,1.323893,0.402156,0.250005,0.914159,1.944965,1.775579,0.057447,1.570132,...,-0.587471,-1.137527,0.867144,0.874174,-1.046653,1.061666,0.006723,0.818896,0.158048,0.591951
2,5,3.882333,3.737990,2.838781,2.439697,2.106332,3.693159,2.954819,-1.532544,3.316317,...,0.382979,0.327910,0.317124,0.314843,0.955835,-0.416433,-0.005506,0.030302,0.491466,0.125287
3,6,0.272010,0.243292,-0.188322,-0.173267,-0.298691,0.539981,1.030152,-0.588824,0.086587,...,0.206509,0.302809,-0.043210,-0.039529,-1.046653,-1.325080,-0.571561,-0.595299,-0.343105,-0.171810
4,7,-0.279947,-0.276768,-0.267334,-0.246457,-0.121783,-0.389423,-0.276634,-1.219836,-0.385013,...,-0.352575,-0.802031,-0.380093,-0.356896,-1.045213,-0.312954,-0.913789,-0.525402,-1.044475,-0.937489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,-0.363330,-0.338285,-0.245789,-0.215434,0.514834,-0.329374,-0.302795,-1.518032,-0.282195,...,0.449480,-0.686836,0.754932,0.755514,0.955835,1.326664,-0.029336,0.835049,0.483801,0.912667
790,996,0.086315,-0.017169,0.034644,-0.023899,1.035685,0.172986,-0.430163,-0.359064,0.915107,...,0.724510,-0.232904,0.322665,0.323636,-1.046653,0.711623,0.722250,0.657643,0.440393,-0.051937
791,997,-0.415523,-0.404640,-0.301064,-0.272261,-0.283553,-0.426545,-0.319149,-1.384230,-0.480706,...,0.630059,0.768304,-0.366326,-0.366658,0.955835,-0.994472,-0.001209,0.021466,-0.183229,0.362709
792,999,-0.374264,-0.374968,-0.307572,-0.280468,-0.389303,-0.288232,-0.210823,0.668369,-0.367042,...,-0.375600,-0.050453,-0.950951,-0.952429,0.955835,-0.114814,-0.507072,-0.264246,-0.549834,-0.019380


Essentia Best Overall + openSMILE emobase

In [66]:
# Fit and transform the selected columns
df_pmemo_integrated_essentia_best_overall_opensmile_emobase_standardised = pd.DataFrame(scaler.fit_transform(df_pmemo_integrated_essentia_best_overall_opensmile_emobase), columns=df_pmemo_integrated_essentia_best_overall_opensmile_emobase.columns)

df_pmemo_integrated_essentia_best_overall_opensmile_emobase_standardised = df_pmemo_integrated_essentia_best_overall_opensmile_emobase_standardised.drop('song_id', axis=1)
df_pmemo_integrated_essentia_best_overall_opensmile_emobase_standardised.insert(0, column='song_id', value=song_ids)

df_pmemo_integrated_essentia_best_overall_opensmile_emobase_standardised.to_csv(get_pmemo_path('processed/features/integrated/standardised_essentia_best_overall_opensmile_emobase_features.csv'))

df_pmemo_integrated_essentia_best_overall_opensmile_emobase_standardised

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
0,1,-0.212798,-0.221286,-0.207779,-0.192209,0.097903,-0.282657,-0.373510,-0.792719,-0.098023,...,0.617745,0.712242,-0.243054,-0.374326,-0.498373,0.067263,0.357042,0.501149,0.354853,0.431765
1,4,1.398278,1.323893,0.402156,0.250005,0.914159,1.944965,1.775579,0.057447,1.570132,...,-1.234588,-1.434384,-0.683549,-0.056562,0.708067,0.067263,-0.670308,-0.709353,-0.669416,-0.694541
2,5,3.882333,3.737990,2.838781,2.439697,2.106332,3.693159,2.954819,-1.532544,3.316317,...,1.363365,1.325873,-0.821297,-0.722094,-1.406147,0.067263,0.950732,1.411980,0.946761,1.190031
3,6,0.272010,0.243292,-0.188322,-0.173267,-0.298691,0.539981,1.030152,-0.588824,0.086587,...,2.433465,2.100930,-0.974874,-0.760861,-2.953295,0.067263,3.428463,2.964336,3.417057,3.211612
4,7,-0.279947,-0.276768,-0.267334,-0.246457,-0.121783,-0.389423,-0.276634,-1.219836,-0.385013,...,-0.256993,-0.141904,-0.048648,-0.192448,0.532217,0.067263,-0.599592,-0.532911,-0.598911,-0.569687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,-0.363330,-0.338285,-0.245789,-0.215434,0.514834,-0.329374,-0.302795,-1.518032,-0.282195,...,0.269117,0.391591,-0.532797,-0.360134,0.637909,0.067263,-0.028897,-0.638959,-0.029929,-0.339677
790,996,0.086315,-0.017169,0.034644,-0.023899,1.035685,0.172986,-0.430163,-0.359064,0.915107,...,-0.232666,-0.114583,-0.894306,-0.626096,0.708067,0.067263,-0.670308,-0.709353,-0.669416,-0.694541
791,997,-0.415523,-0.404640,-0.301064,-0.272261,-0.283553,-0.426545,-0.319149,-1.384230,-0.480706,...,-0.233035,-0.116966,-0.144450,-0.249582,0.209247,0.067263,-0.470457,-0.208853,-0.470164,-0.340747
792,999,-0.374264,-0.374968,-0.307572,-0.280468,-0.389303,-0.288232,-0.210823,0.668369,-0.367042,...,0.308629,0.429497,-0.638472,-0.456546,0.708067,0.067263,-0.660185,-0.709353,-0.659322,-0.689505


Essentia Best Overall + openSMILE GeMAPS

In [67]:
# Fit and transform the selected columns
df_pmemo_integrated_essentia_best_overall_opensmile_gemaps_standardised = pd.DataFrame(scaler.fit_transform(df_pmemo_integrated_essentia_best_overall_opensmile_gemaps), columns=df_pmemo_integrated_essentia_best_overall_opensmile_gemaps.columns)

df_pmemo_integrated_essentia_best_overall_opensmile_gemaps_standardised = df_pmemo_integrated_essentia_best_overall_opensmile_gemaps_standardised.drop('song_id', axis=1)
df_pmemo_integrated_essentia_best_overall_opensmile_gemaps_standardised.insert(0, column='song_id', value=song_ids)

df_pmemo_integrated_essentia_best_overall_opensmile_gemaps_standardised.to_csv(get_pmemo_path('processed/features/integrated/standardised_essentia_best_overall_opensmile_gemaps_features.csv'))

df_pmemo_integrated_essentia_best_overall_opensmile_gemaps_standardised

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,1,-0.212798,-0.221286,-0.207779,-0.192209,0.097903,-0.282657,-0.373510,-0.792719,-0.098023,...,0.014368,0.110506,-0.359317,0.242259,0.626886,0.384981,-0.331465,-0.379390,0.257481,-0.073466
1,4,1.398278,1.323893,0.402156,0.250005,0.914159,1.944965,1.775579,0.057447,1.570132,...,-2.567809,2.516917,-1.563992,-1.753440,1.245798,-2.045060,0.047275,0.338540,13.752535,12.618872
2,5,3.882333,3.737990,2.838781,2.439697,2.106332,3.693159,2.954819,-1.532544,3.316317,...,-2.379958,2.177954,-1.323570,-0.315244,1.246355,-0.468066,-0.167371,-0.207094,0.501076,0.509763
3,6,0.272010,0.243292,-0.188322,-0.173267,-0.298691,0.539981,1.030152,-0.588824,0.086587,...,-0.734771,0.765704,-1.847202,0.706169,0.561435,-0.917961,-0.425666,-0.426155,4.048747,3.581308
4,7,-0.279947,-0.276768,-0.267334,-0.246457,-0.121783,-0.389423,-0.276634,-1.219836,-0.385013,...,0.471424,-0.688262,1.464579,-0.465351,0.349378,-0.232340,-0.112629,-0.193586,-0.541614,-0.578875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,-0.363330,-0.338285,-0.245789,-0.215434,0.514834,-0.329374,-0.302795,-1.518032,-0.282195,...,0.613497,-0.674903,0.559402,0.318206,-0.269563,0.350924,-0.295113,0.158619,0.007874,-0.106073
790,996,0.086315,-0.017169,0.034644,-0.023899,1.035685,0.172986,-0.430163,-0.359064,0.915107,...,0.504594,-0.120795,0.683018,1.644929,0.507535,0.733571,-0.368790,-0.397402,0.207085,0.135304
791,997,-0.415523,-0.404640,-0.301064,-0.272261,-0.283553,-0.426545,-0.319149,-1.384230,-0.480706,...,0.785874,-0.499345,0.258260,1.052879,-0.034250,0.207191,-0.274159,-0.252014,0.077766,1.235931
792,999,-0.374264,-0.374968,-0.307572,-0.280468,-0.389303,-0.288232,-0.210823,0.668369,-0.367042,...,0.606728,-0.366945,-0.192367,1.193343,-0.846263,0.756339,-0.393204,-0.269206,0.371842,2.290611


Essentia Best Overall + openSMILE eGeMAPS

In [68]:
# Fit and transform the selected columns
df_pmemo_integrated_essentia_best_overall_opensmile_egemaps_standardised = pd.DataFrame(scaler.fit_transform(df_pmemo_integrated_essentia_best_overall_opensmile_egemaps), columns=df_pmemo_integrated_essentia_best_overall_opensmile_egemaps.columns)

df_pmemo_integrated_essentia_best_overall_opensmile_egemaps_standardised = df_pmemo_integrated_essentia_best_overall_opensmile_egemaps_standardised.drop('song_id', axis=1)
df_pmemo_integrated_essentia_best_overall_opensmile_egemaps_standardised.insert(0, column='song_id', value=song_ids)

df_pmemo_integrated_essentia_best_overall_opensmile_egemaps_standardised.to_csv(get_pmemo_path('processed/features/integrated/standardised_essentia_best_overall_opensmile_egemaps_features.csv'))

df_pmemo_integrated_essentia_best_overall_opensmile_egemaps_standardised

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,1,-0.212798,-0.221286,-0.207779,-0.192209,0.097903,-0.282657,-0.373510,-0.792719,-0.098023,...,-0.359317,0.242259,-1.044193,0.626886,0.384981,-0.331465,-0.379390,0.257481,-0.073466,-1.152246
1,4,1.398278,1.323893,0.402156,0.250005,0.914159,1.944965,1.775579,0.057447,1.570132,...,-1.563992,-1.753440,2.568822,1.245798,-2.045060,0.047275,0.338540,13.752535,12.618872,1.003751
2,5,3.882333,3.737990,2.838781,2.439697,2.106332,3.693159,2.954819,-1.532544,3.316317,...,-1.323570,-0.315244,-0.328482,1.246355,-0.468066,-0.167371,-0.207094,0.501076,0.509763,0.430054
3,6,0.272010,0.243292,-0.188322,-0.173267,-0.298691,0.539981,1.030152,-0.588824,0.086587,...,-1.847202,0.706169,-0.972752,0.561435,-0.917961,-0.425666,-0.426155,4.048747,3.581308,0.433514
4,7,-0.279947,-0.276768,-0.267334,-0.246457,-0.121783,-0.389423,-0.276634,-1.219836,-0.385013,...,1.464579,-0.465351,-0.454355,0.349378,-0.232340,-0.112629,-0.193586,-0.541614,-0.578875,0.390891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,-0.363330,-0.338285,-0.245789,-0.215434,0.514834,-0.329374,-0.302795,-1.518032,-0.282195,...,0.559402,0.318206,0.230459,-0.269563,0.350924,-0.295113,0.158619,0.007874,-0.106073,0.385753
790,996,0.086315,-0.017169,0.034644,-0.023899,1.035685,0.172986,-0.430163,-0.359064,0.915107,...,0.683018,1.644929,0.184761,0.507535,0.733571,-0.368790,-0.397402,0.207085,0.135304,0.646084
791,997,-0.415523,-0.404640,-0.301064,-0.272261,-0.283553,-0.426545,-0.319149,-1.384230,-0.480706,...,0.258260,1.052879,-0.660618,-0.034250,0.207191,-0.274159,-0.252014,0.077766,1.235931,0.392324
792,999,-0.374264,-0.374968,-0.307572,-0.280468,-0.389303,-0.288232,-0.210823,0.668369,-0.367042,...,-0.192367,1.193343,0.310736,-0.846263,0.756339,-0.393204,-0.269206,0.371842,2.290611,0.968205


### Data Normalisation

In [69]:
scaler = MinMaxScaler()

Essentia All + openSMILE ComParE2016

In [70]:
# Fit and transform the selected columns
df_pmemo_integrated_essentia_all_opensmile_compare2016_normalised = pd.DataFrame(scaler.fit_transform(df_pmemo_integrated_essentia_all_opensmile_compare2016), columns=df_pmemo_integrated_essentia_all_opensmile_compare2016.columns)

df_pmemo_integrated_essentia_all_opensmile_compare2016_normalised = df_pmemo_integrated_essentia_all_opensmile_compare2016_normalised.drop('song_id', axis=1)
df_pmemo_integrated_essentia_all_opensmile_compare2016_normalised.insert(0, column='song_id', value=song_ids)

df_pmemo_integrated_essentia_all_opensmile_compare2016_normalised.to_csv(get_pmemo_path('processed/features/integrated/normalised_essentia_all_opensmile_compare2016_features.csv'))

df_pmemo_integrated_essentia_all_opensmile_compare2016_normalised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,1,0.984985,0.300101,0.248777,0.343661,0.308275,0.845139,0.399331,0.349027,0.207137,...,0.329262,0.399650,0.392199,0.392725,0.000000e+00,0.617706,0.404827,0.296608,0.351489,0.401564
1,4,0.952336,0.246265,0.263136,0.214000,0.271672,0.701362,0.583063,0.580314,0.715080,...,0.202546,0.304287,0.395144,0.395924,2.750000e-07,0.619267,0.345959,0.485218,0.316289,0.365470
2,5,0.944813,0.302274,0.252729,0.578914,0.500009,0.974658,0.842132,0.789327,0.304432,...,0.322991,0.547101,0.326090,0.325708,1.000000e+00,0.431744,0.344214,0.363787,0.362612,0.299804
3,6,0.968798,0.095359,0.039680,0.199956,0.104916,0.790293,0.774950,0.792841,0.142098,...,0.301089,0.542942,0.280851,0.281222,0.000000e+00,0.316465,0.263451,0.267455,0.246664,0.257999
4,7,0.985253,0.500611,0.451491,0.658070,0.619426,0.864781,0.318126,0.243341,0.182706,...,0.231700,0.359877,0.238556,0.241381,7.192500e-04,0.444872,0.214624,0.278218,0.149222,0.150258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.914366,0.443439,0.416705,0.424372,0.486538,0.774487,0.386337,0.347959,0.116537,...,0.331244,0.378964,0.381056,0.381028,1.000000e+00,0.652887,0.340814,0.487705,0.361547,0.410599
790,996,0.937592,0.419159,0.402542,0.522361,0.507475,0.835013,0.333683,0.209687,0.090479,...,0.365379,0.454177,0.326786,0.326812,0.000000e+00,0.574858,0.448047,0.460387,0.355516,0.274867
791,997,0.927177,0.398546,0.383329,0.412846,0.518133,0.895981,0.428588,0.396050,0.166951,...,0.353656,0.620072,0.240284,0.240155,1.000000e+00,0.358409,0.344827,0.362427,0.268875,0.333213
792,999,0.969786,0.411355,0.311743,0.273333,0.207960,0.866638,0.398838,0.295906,0.248863,...,0.228842,0.484409,0.166886,0.166620,1.000000e+00,0.470009,0.272652,0.318432,0.217943,0.279448


Essentia All + openSMILE emobase

In [71]:
# Fit and transform the selected columns
df_pmemo_integrated_essentia_all_opensmile_emobase_normalised = pd.DataFrame(scaler.fit_transform(df_pmemo_integrated_essentia_all_opensmile_emobase), columns=df_pmemo_integrated_essentia_all_opensmile_emobase.columns)

df_pmemo_integrated_essentia_all_opensmile_emobase_normalised = df_pmemo_integrated_essentia_all_opensmile_emobase_normalised.drop('song_id', axis=1)
df_pmemo_integrated_essentia_all_opensmile_emobase_normalised.insert(0, column='song_id', value=song_ids)

df_pmemo_integrated_essentia_all_opensmile_emobase_normalised.to_csv(get_pmemo_path('processed/features/integrated/normalised_essentia_all_opensmile_emobase_features.csv'))

df_pmemo_integrated_essentia_all_opensmile_emobase_normalised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
0,1,0.984985,0.300101,0.248777,0.343661,0.308275,0.845139,0.399331,0.349027,0.207137,...,0.403700,0.588364,0.125589,0.047908,0.758451,1.0,0.203203,0.243753,0.203203,0.229604
1,4,0.952336,0.246265,0.263136,0.214000,0.271672,0.701362,0.583063,0.580314,0.715080,...,0.096648,0.233917,0.067041,0.087292,1.000000,1.0,0.000000,0.000000,0.000000,0.000000
2,5,0.944813,0.302274,0.252729,0.578914,0.500009,0.974658,0.842132,0.789327,0.304432,...,0.527297,0.689686,0.048732,0.004805,0.576701,1.0,0.320631,0.427163,0.320631,0.384180
3,6,0.968798,0.095359,0.039680,0.199956,0.104916,0.790293,0.774950,0.792841,0.142098,...,0.704682,0.817662,0.028320,0.000000,0.266937,1.0,0.810709,0.739754,0.810709,0.796290
4,7,0.985253,0.500611,0.451491,0.658070,0.619426,0.864781,0.318126,0.243341,0.182706,...,0.258699,0.447329,0.151428,0.070450,0.964792,1.0,0.013987,0.035529,0.013987,0.025452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.914366,0.443439,0.416705,0.424372,0.486538,0.774487,0.386337,0.347959,0.116537,...,0.345910,0.535419,0.087078,0.049667,0.985953,1.0,0.126867,0.014175,0.126867,0.072341
790,996,0.937592,0.419159,0.402542,0.522361,0.507475,0.835013,0.333683,0.209687,0.090479,...,0.262732,0.451840,0.039029,0.016703,1.000000,1.0,0.000000,0.000000,0.000000,0.000000
791,997,0.927177,0.398546,0.383329,0.412846,0.518133,0.895981,0.428588,0.396050,0.166951,...,0.262670,0.451447,0.138694,0.063369,0.900128,1.0,0.039529,0.100783,0.039529,0.072123
792,999,0.969786,0.411355,0.311743,0.273333,0.207960,0.866638,0.398838,0.295906,0.248863,...,0.352459,0.541678,0.073032,0.037717,1.000000,1.0,0.002002,0.000000,0.002002,0.001027


Essentia All + openSMILE GeMAPs

In [72]:
# Fit and transform the selected columns
df_pmemo_integrated_essentia_all_opensmile_gemaps_normalised = pd.DataFrame(scaler.fit_transform(df_pmemo_integrated_essentia_all_opensmile_gemaps), columns=df_pmemo_integrated_essentia_all_opensmile_gemaps.columns)

df_pmemo_integrated_essentia_all_opensmile_gemaps_normalised = df_pmemo_integrated_essentia_all_opensmile_gemaps_normalised.drop('song_id', axis=1)
df_pmemo_integrated_essentia_all_opensmile_gemaps_normalised.insert(0, column='song_id', value=song_ids)

df_pmemo_integrated_essentia_all_opensmile_gemaps_normalised.to_csv(get_pmemo_path('processed/features/integrated/normalised_essentia_all_opensmile_gemaps_features.csv'))

df_pmemo_integrated_essentia_all_opensmile_gemaps_normalised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,1,0.984985,0.300101,0.248777,0.343661,0.308275,0.845139,0.399331,0.349027,0.207137,...,0.655572,0.388723,0.344370,0.493393,0.618233,0.568024,0.012333,0.010856,0.092155,0.066829
1,4,0.952336,0.246265,0.263136,0.214000,0.271672,0.701362,0.583063,0.580314,0.715080,...,0.188228,0.814309,0.185585,0.222545,0.719396,0.075804,0.038816,0.059588,1.000000,1.000000
2,5,0.944813,0.302274,0.252729,0.578914,0.500009,0.974658,0.842132,0.789327,0.304432,...,0.222227,0.754362,0.217274,0.417731,0.719487,0.395234,0.023807,0.022551,0.108542,0.109709
3,6,0.968798,0.095359,0.039680,0.199956,0.104916,0.790293,0.774950,0.792841,0.142098,...,0.519987,0.504598,0.148255,0.556353,0.607535,0.304105,0.005747,0.007682,0.347203,0.335536
4,7,0.985253,0.500611,0.451491,0.658070,0.619426,0.864781,0.318126,0.243341,0.182706,...,0.738294,0.247457,0.584773,0.397359,0.572874,0.442982,0.027635,0.023468,0.038398,0.029670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.914366,0.443439,0.416705,0.424372,0.486538,0.774487,0.386337,0.347959,0.116537,...,0.764008,0.249820,0.465464,0.503700,0.471706,0.561126,0.014875,0.047375,0.075363,0.064431
790,996,0.937592,0.419159,0.402542,0.522361,0.507475,0.835013,0.333683,0.209687,0.090479,...,0.744298,0.347816,0.481757,0.683758,0.598725,0.638633,0.009723,0.009633,0.088765,0.082178
791,997,0.927177,0.398546,0.383329,0.412846,0.518133,0.895981,0.428588,0.396050,0.166951,...,0.795206,0.280868,0.425771,0.603407,0.510169,0.532011,0.016340,0.019502,0.080065,0.163099
792,999,0.969786,0.411355,0.311743,0.273333,0.207960,0.866638,0.398838,0.295906,0.248863,...,0.762783,0.304284,0.366375,0.622470,0.377443,0.643245,0.008016,0.018335,0.099848,0.240641


Essentia All + openSMILE eGeMAPS

In [73]:
# Fit and transform the selected columns
df_pmemo_integrated_essentia_all_opensmile_egemaps_normalised = pd.DataFrame(scaler.fit_transform(df_pmemo_integrated_essentia_all_opensmile_egemaps), columns=df_pmemo_integrated_essentia_all_opensmile_egemaps.columns)

df_pmemo_integrated_essentia_all_opensmile_egemaps_normalised = df_pmemo_integrated_essentia_all_opensmile_egemaps_normalised.drop('song_id', axis=1)
df_pmemo_integrated_essentia_all_opensmile_egemaps_normalised.insert(0, column='song_id', value=song_ids)

df_pmemo_integrated_essentia_all_opensmile_egemaps_normalised.to_csv(get_pmemo_path('processed/features/integrated/normalised_essentia_all_opensmile_egemaps_features.csv'))

df_pmemo_integrated_essentia_all_opensmile_egemaps_normalised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,1,0.984985,0.300101,0.248777,0.343661,0.308275,0.845139,0.399331,0.349027,0.207137,...,0.344370,0.493393,0.355293,0.618233,0.568024,0.012333,0.010856,0.092155,0.066829,0.644904
1,4,0.952336,0.246265,0.263136,0.214000,0.271672,0.701362,0.583063,0.580314,0.715080,...,0.185585,0.222545,0.997079,0.719396,0.075804,0.038816,0.059588,1.000000,1.000000,0.893336
2,5,0.944813,0.302274,0.252729,0.578914,0.500009,0.974658,0.842132,0.789327,0.304432,...,0.217274,0.417731,0.482426,0.719487,0.395234,0.023807,0.022551,0.108542,0.109709,0.827230
3,6,0.968798,0.095359,0.039680,0.199956,0.104916,0.790293,0.774950,0.792841,0.142098,...,0.148255,0.556353,0.367983,0.607535,0.304105,0.005747,0.007682,0.347203,0.335536,0.827628
4,7,0.985253,0.500611,0.451491,0.658070,0.619426,0.864781,0.318126,0.243341,0.182706,...,0.584773,0.397359,0.460067,0.572874,0.442982,0.027635,0.023468,0.038398,0.029670,0.822717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.914366,0.443439,0.416705,0.424372,0.486538,0.774487,0.386337,0.347959,0.116537,...,0.465464,0.503700,0.581712,0.471706,0.561126,0.014875,0.047375,0.075363,0.064431,0.822125
790,996,0.937592,0.419159,0.402542,0.522361,0.507475,0.835013,0.333683,0.209687,0.090479,...,0.481757,0.683758,0.573594,0.598725,0.638633,0.009723,0.009633,0.088765,0.082178,0.852123
791,997,0.927177,0.398546,0.383329,0.412846,0.518133,0.895981,0.428588,0.396050,0.166951,...,0.425771,0.603407,0.423428,0.510169,0.532011,0.016340,0.019502,0.080065,0.163099,0.822882
792,999,0.969786,0.411355,0.311743,0.273333,0.207960,0.866638,0.398838,0.295906,0.248863,...,0.366375,0.622470,0.595971,0.377443,0.643245,0.008016,0.018335,0.099848,0.240641,0.889240


Essentia Best Overall + openSMILE ComParE2016

In [74]:
# Fit and transform the selected columns
df_pmemo_integrated_essentia_best_overall_opensmile_compare2016_normalised = pd.DataFrame(scaler.fit_transform(df_pmemo_integrated_essentia_best_overall_opensmile_compare2016), columns=df_pmemo_integrated_essentia_best_overall_opensmile_compare2016.columns)

df_pmemo_integrated_essentia_best_overall_opensmile_compare2016_normalised = df_pmemo_integrated_essentia_best_overall_opensmile_compare2016_normalised.drop('song_id', axis=1)
df_pmemo_integrated_essentia_best_overall_opensmile_compare2016_normalised.insert(0, column='song_id', value=song_ids)

df_pmemo_integrated_essentia_best_overall_opensmile_compare2016_normalised.to_csv(get_pmemo_path('processed/features/integrated/normalised_essentia_best_overall_opensmile_compare2016_features.csv'))

df_pmemo_integrated_essentia_best_overall_opensmile_compare2016_normalised

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,1,0.057462,0.052210,0.011674,0.009177,0.084971,0.052158,0.021677,0.200273,0.111279,...,0.329262,0.399650,0.392199,0.392725,0.000000e+00,0.617706,0.404827,0.296608,0.351489,0.401564
1,4,0.229843,0.210377,0.067982,0.045153,0.166129,0.279947,0.152932,0.408428,0.373784,...,0.202546,0.304287,0.395144,0.395924,2.750000e-07,0.619267,0.345959,0.485218,0.316289,0.365470
2,5,0.495631,0.457488,0.292928,0.223292,0.284663,0.458712,0.224953,0.019133,0.648568,...,0.322991,0.547101,0.326090,0.325708,1.000000e+00,0.431744,0.344214,0.363787,0.362612,0.299804
3,6,0.109335,0.099765,0.013470,0.010718,0.045539,0.136278,0.107405,0.250194,0.140329,...,0.301089,0.542942,0.280851,0.281222,0.000000e+00,0.316465,0.263451,0.267455,0.246664,0.257999
4,7,0.050277,0.046530,0.006176,0.004764,0.063129,0.041240,0.027593,0.095697,0.066117,...,0.231700,0.359877,0.238556,0.241381,7.192500e-04,0.444872,0.214624,0.278218,0.149222,0.150258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.041355,0.040233,0.008165,0.007288,0.126425,0.047381,0.025996,0.022686,0.082297,...,0.331244,0.378964,0.381056,0.381028,1.000000e+00,0.652887,0.340814,0.487705,0.361547,0.410599
790,996,0.089466,0.073103,0.034054,0.022870,0.178212,0.098750,0.018217,0.306449,0.270708,...,0.365379,0.454177,0.326786,0.326812,0.000000e+00,0.574858,0.448047,0.460387,0.355516,0.274867
791,997,0.035771,0.033441,0.003062,0.002665,0.047044,0.037444,0.024997,0.055447,0.051059,...,0.353656,0.620072,0.240284,0.240155,1.000000e+00,0.358409,0.344827,0.362427,0.268875,0.333213
792,999,0.040185,0.036478,0.002461,0.001997,0.036530,0.051588,0.031613,0.558007,0.068945,...,0.228842,0.484409,0.166886,0.166620,1.000000e+00,0.470009,0.272652,0.318432,0.217943,0.279448


Essentia Best Overall + openSMILE emobase

In [75]:
# Fit and transform the selected columns
df_pmemo_integrated_essentia_best_overall_opensmile_emobase_normalised = pd.DataFrame(scaler.fit_transform(df_pmemo_integrated_essentia_best_overall_opensmile_emobase), columns=df_pmemo_integrated_essentia_best_overall_opensmile_emobase.columns)

df_pmemo_integrated_essentia_best_overall_opensmile_emobase_normalised = df_pmemo_integrated_essentia_best_overall_opensmile_emobase_normalised.drop('song_id', axis=1)
df_pmemo_integrated_essentia_best_overall_opensmile_emobase_normalised.insert(0, column='song_id', value=song_ids)

df_pmemo_integrated_essentia_best_overall_opensmile_emobase_normalised.to_csv(get_pmemo_path('processed/features/integrated/normalised_essentia_best_overall_opensmile_emobase_features.csv'))

df_pmemo_integrated_essentia_best_overall_opensmile_emobase_normalised

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,F0env_sma_de_linregerrQ,F0env_sma_de_stddev,F0env_sma_de_skewness,F0env_sma_de_kurtosis,F0env_sma_de_quartile1,F0env_sma_de_quartile2,F0env_sma_de_quartile3,F0env_sma_de_iqr1-2,F0env_sma_de_iqr2-3,F0env_sma_de_iqr1-3
0,1,0.057462,0.052210,0.011674,0.009177,0.084971,0.052158,0.021677,0.200273,0.111279,...,0.403700,0.588364,0.125589,0.047908,0.758451,1.0,0.203203,0.243753,0.203203,0.229604
1,4,0.229843,0.210377,0.067982,0.045153,0.166129,0.279947,0.152932,0.408428,0.373784,...,0.096648,0.233917,0.067041,0.087292,1.000000,1.0,0.000000,0.000000,0.000000,0.000000
2,5,0.495631,0.457488,0.292928,0.223292,0.284663,0.458712,0.224953,0.019133,0.648568,...,0.527297,0.689686,0.048732,0.004805,0.576701,1.0,0.320631,0.427163,0.320631,0.384180
3,6,0.109335,0.099765,0.013470,0.010718,0.045539,0.136278,0.107405,0.250194,0.140329,...,0.704682,0.817662,0.028320,0.000000,0.266937,1.0,0.810709,0.739754,0.810709,0.796290
4,7,0.050277,0.046530,0.006176,0.004764,0.063129,0.041240,0.027593,0.095697,0.066117,...,0.258699,0.447329,0.151428,0.070450,0.964792,1.0,0.013987,0.035529,0.013987,0.025452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.041355,0.040233,0.008165,0.007288,0.126425,0.047381,0.025996,0.022686,0.082297,...,0.345910,0.535419,0.087078,0.049667,0.985953,1.0,0.126867,0.014175,0.126867,0.072341
790,996,0.089466,0.073103,0.034054,0.022870,0.178212,0.098750,0.018217,0.306449,0.270708,...,0.262732,0.451840,0.039029,0.016703,1.000000,1.0,0.000000,0.000000,0.000000,0.000000
791,997,0.035771,0.033441,0.003062,0.002665,0.047044,0.037444,0.024997,0.055447,0.051059,...,0.262670,0.451447,0.138694,0.063369,0.900128,1.0,0.039529,0.100783,0.039529,0.072123
792,999,0.040185,0.036478,0.002461,0.001997,0.036530,0.051588,0.031613,0.558007,0.068945,...,0.352459,0.541678,0.073032,0.037717,1.000000,1.0,0.002002,0.000000,0.002002,0.001027


Essentia Best Overall + openSMILE GeMAPS

In [76]:
# Fit and transform the selected columns
df_pmemo_integrated_essentia_best_overall_opensmile_gemaps_normalised = pd.DataFrame(scaler.fit_transform(df_pmemo_integrated_essentia_best_overall_opensmile_gemaps), columns=df_pmemo_integrated_essentia_best_overall_opensmile_gemaps.columns)

df_pmemo_integrated_essentia_best_overall_opensmile_gemaps_normalised = df_pmemo_integrated_essentia_best_overall_opensmile_gemaps_normalised.drop('song_id', axis=1)
df_pmemo_integrated_essentia_best_overall_opensmile_gemaps_normalised.insert(0, column='song_id', value=song_ids)

df_pmemo_integrated_essentia_best_overall_opensmile_gemaps_normalised.to_csv(get_pmemo_path('processed/features/integrated/normalised_essentia_best_overall_opensmile_gemaps_features.csv'))

df_pmemo_integrated_essentia_best_overall_opensmile_gemaps_normalised

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,alphaRatioUV_sma3nz_amean,hammarbergIndexUV_sma3nz_amean,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength
0,1,0.057462,0.052210,0.011674,0.009177,0.084971,0.052158,0.021677,0.200273,0.111279,...,0.655572,0.388723,0.344370,0.493393,0.618233,0.568024,0.012333,0.010856,0.092155,0.066829
1,4,0.229843,0.210377,0.067982,0.045153,0.166129,0.279947,0.152932,0.408428,0.373784,...,0.188228,0.814309,0.185585,0.222545,0.719396,0.075804,0.038816,0.059588,1.000000,1.000000
2,5,0.495631,0.457488,0.292928,0.223292,0.284663,0.458712,0.224953,0.019133,0.648568,...,0.222227,0.754362,0.217274,0.417731,0.719487,0.395234,0.023807,0.022551,0.108542,0.109709
3,6,0.109335,0.099765,0.013470,0.010718,0.045539,0.136278,0.107405,0.250194,0.140329,...,0.519987,0.504598,0.148255,0.556353,0.607535,0.304105,0.005747,0.007682,0.347203,0.335536
4,7,0.050277,0.046530,0.006176,0.004764,0.063129,0.041240,0.027593,0.095697,0.066117,...,0.738294,0.247457,0.584773,0.397359,0.572874,0.442982,0.027635,0.023468,0.038398,0.029670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.041355,0.040233,0.008165,0.007288,0.126425,0.047381,0.025996,0.022686,0.082297,...,0.764008,0.249820,0.465464,0.503700,0.471706,0.561126,0.014875,0.047375,0.075363,0.064431
790,996,0.089466,0.073103,0.034054,0.022870,0.178212,0.098750,0.018217,0.306449,0.270708,...,0.744298,0.347816,0.481757,0.683758,0.598725,0.638633,0.009723,0.009633,0.088765,0.082178
791,997,0.035771,0.033441,0.003062,0.002665,0.047044,0.037444,0.024997,0.055447,0.051059,...,0.795206,0.280868,0.425771,0.603407,0.510169,0.532011,0.016340,0.019502,0.080065,0.163099
792,999,0.040185,0.036478,0.002461,0.001997,0.036530,0.051588,0.031613,0.558007,0.068945,...,0.762783,0.304284,0.366375,0.622470,0.377443,0.643245,0.008016,0.018335,0.099848,0.240641


Essentia Best Overall + openSMILE eGeMAPS

In [77]:
# Fit and transform the selected columns
df_pmemo_integrated_essentia_best_overall_opensmile_egemaps_normalised = pd.DataFrame(scaler.fit_transform(df_pmemo_integrated_essentia_best_overall_opensmile_egemaps), columns=df_pmemo_integrated_essentia_best_overall_opensmile_egemaps.columns)

df_pmemo_integrated_essentia_best_overall_opensmile_egemaps_normalised = df_pmemo_integrated_essentia_best_overall_opensmile_egemaps_normalised.drop('song_id', axis=1)
df_pmemo_integrated_essentia_best_overall_opensmile_egemaps_normalised.insert(0, column='song_id', value=song_ids)

df_pmemo_integrated_essentia_best_overall_opensmile_egemaps_normalised.to_csv(get_pmemo_path('processed/features/integrated/normalised_essentia_best_overall_opensmile_egemaps_features.csv'))

df_pmemo_integrated_essentia_best_overall_opensmile_egemaps_normalised

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,1,0.057462,0.052210,0.011674,0.009177,0.084971,0.052158,0.021677,0.200273,0.111279,...,0.344370,0.493393,0.355293,0.618233,0.568024,0.012333,0.010856,0.092155,0.066829,0.644904
1,4,0.229843,0.210377,0.067982,0.045153,0.166129,0.279947,0.152932,0.408428,0.373784,...,0.185585,0.222545,0.997079,0.719396,0.075804,0.038816,0.059588,1.000000,1.000000,0.893336
2,5,0.495631,0.457488,0.292928,0.223292,0.284663,0.458712,0.224953,0.019133,0.648568,...,0.217274,0.417731,0.482426,0.719487,0.395234,0.023807,0.022551,0.108542,0.109709,0.827230
3,6,0.109335,0.099765,0.013470,0.010718,0.045539,0.136278,0.107405,0.250194,0.140329,...,0.148255,0.556353,0.367983,0.607535,0.304105,0.005747,0.007682,0.347203,0.335536,0.827628
4,7,0.050277,0.046530,0.006176,0.004764,0.063129,0.041240,0.027593,0.095697,0.066117,...,0.584773,0.397359,0.460067,0.572874,0.442982,0.027635,0.023468,0.038398,0.029670,0.822717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.041355,0.040233,0.008165,0.007288,0.126425,0.047381,0.025996,0.022686,0.082297,...,0.465464,0.503700,0.581712,0.471706,0.561126,0.014875,0.047375,0.075363,0.064431,0.822125
790,996,0.089466,0.073103,0.034054,0.022870,0.178212,0.098750,0.018217,0.306449,0.270708,...,0.481757,0.683758,0.573594,0.598725,0.638633,0.009723,0.009633,0.088765,0.082178,0.852123
791,997,0.035771,0.033441,0.003062,0.002665,0.047044,0.037444,0.024997,0.055447,0.051059,...,0.425771,0.603407,0.423428,0.510169,0.532011,0.016340,0.019502,0.080065,0.163099,0.822882
792,999,0.040185,0.036478,0.002461,0.001997,0.036530,0.051588,0.031613,0.558007,0.068945,...,0.366375,0.622470,0.595971,0.377443,0.643245,0.008016,0.018335,0.099848,0.240641,0.889240


## Filter out only the mean features

In [78]:
df_essentia_best_overall_features

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,1,12.878959,19.579144,1312.717041,3060.723389,639.860474,19.044914,5.928576,-1.777708,44.479679,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.630969,0.000000,10.998553,7.814761,18.958033
1,4,47.323818,71.870140,7632.717773,15035.108398,1229.549561,95.002518,40.996933,-1.586174,143.324692,...,0.991735,4.793388,25.785124,0.000000,0.661157,6.115703,0.000000,5.289256,0.000000,10.247934
2,5,100.432968,153.566513,32880.425781,74328.078125,2090.814209,154.612579,60.239502,-1.944383,246.793365,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.441745,0.220872,0.000000,0.000000,0.000000
3,6,23.244177,35.301132,1514.325195,3573.641113,353.348297,47.095280,28.833210,-1.731773,55.418545,...,0.000000,3.930818,0.000000,0.000000,0.000000,0.000000,0.000000,2.515723,0.000000,0.000000
4,7,11.443307,17.701553,695.618958,1591.793823,481.152069,15.404405,7.509385,-1.873933,27.474340,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.017544,13.078150,6.379585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,9.660586,15.619761,918.862549,2431.855225,941.064697,17.451988,7.082492,-1.941114,33.566727,...,0.000000,0.000000,0.000000,2.113821,0.000000,0.000000,0.000000,0.650406,12.357723,56.829270
790,996,19.273998,26.486744,3824.638184,7618.275879,1317.343872,34.581482,5.004123,-1.680010,104.511772,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.521780,1.112141
791,997,8.544683,13.374214,346.121216,893.072449,364.284393,14.138631,6.815628,-1.910969,21.804148,...,0.479616,1.598721,0.000000,3.597122,0.000000,1.438849,0.000000,7.354117,2.797762,9.832134
792,999,9.426803,14.378345,278.683746,670.824463,287.886932,18.854839,8.583271,-1.448540,28.539162,...,0.000000,0.000000,0.000000,0.000000,0.118765,0.000000,1.543943,0.000000,8.907364,1.306413


In [79]:
col_names = df_essentia_best_overall_features.columns.to_list()
feature_mean_cols = [col for col in col_names if 'dmean' not in col and 'dmean2' not in col and 'dvar' not in col and 'dvar2' not in col and 'max' not in col and 'median' not in col and 'min' not in col and 'stdev' not in col and 'var' not in col]
print(feature_mean_cols)

['song_id', 'lowlevel.melbands_kurtosis.mean', 'lowlevel.melbands_skewness.mean', 'lowlevel.spectral_energy.mean', 'tonal.chords_strength.mean', 'tonal.hpcp_entropy.mean', 'tonal.key_edma.strength', 'tonal.key_temperley.strength', 'rhythm.beats_loudness_band_ratio.mean_0', 'rhythm.beats_loudness_band_ratio.mean_1', 'rhythm.beats_loudness_band_ratio.mean_2', 'rhythm.beats_loudness_band_ratio.mean_3', 'rhythm.beats_loudness_band_ratio.mean_4', 'rhythm.beats_loudness_band_ratio.mean_5', 'tonal.chords_histogram_0', 'tonal.chords_histogram_1', 'tonal.chords_histogram_2', 'tonal.chords_histogram_3', 'tonal.chords_histogram_4', 'tonal.chords_histogram_5', 'tonal.chords_histogram_6', 'tonal.chords_histogram_7', 'tonal.chords_histogram_8', 'tonal.chords_histogram_9', 'tonal.chords_histogram_10', 'tonal.chords_histogram_11', 'tonal.chords_histogram_12', 'tonal.chords_histogram_13', 'tonal.chords_histogram_14', 'tonal.chords_histogram_15', 'tonal.chords_histogram_16', 'tonal.chords_histogram_17',

In [80]:
df_essentia_best_overall_features_mean = df_essentia_best_overall_features[feature_mean_cols]
df_essentia_best_overall_features_mean

,song_id,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.spectral_energy.mean,tonal.chords_strength.mean,tonal.hpcp_entropy.mean,tonal.key_edma.strength,tonal.key_temperley.strength,rhythm.beats_loudness_band_ratio.mean_0,rhythm.beats_loudness_band_ratio.mean_1,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,1,19.044914,2.916830,0.030159,0.483265,1.773851,0.647135,0.678089,0.414544,0.056752,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.630969,0.000000,10.998553,7.814761,18.958033
1,4,95.002518,6.664732,0.173342,0.495532,1.739260,0.427637,0.442323,0.687195,0.074914,...,0.991735,4.793388,25.785124,0.000000,0.661157,6.115703,0.000000,5.289256,0.000000,10.247934
2,5,154.612579,7.777407,0.099738,0.548106,1.986469,0.754484,0.722807,0.713449,0.064303,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.441745,0.220872,0.000000,0.000000,0.000000
3,6,47.095280,5.359872,0.056681,0.514703,1.623544,0.556869,0.529661,0.700700,0.072363,...,0.000000,3.930818,0.000000,0.000000,0.000000,0.000000,0.000000,2.515723,0.000000,0.000000
4,7,15.404405,2.676914,0.043341,0.541958,2.095964,0.773975,0.798527,0.772618,0.059041,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.017544,13.078150,6.379585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,17.451988,2.597063,0.040386,0.578063,1.937530,0.694123,0.727199,0.482837,0.253678,...,0.000000,0.000000,0.000000,2.113821,0.000000,0.000000,0.000000,0.650406,12.357723,56.829270
790,996,34.581482,3.150197,0.039496,0.576943,2.071364,0.709678,0.728921,0.159821,0.219893,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.521780,1.112141
791,997,14.138631,2.758755,0.042173,0.442799,1.921766,0.659779,0.683450,0.474969,0.091173,...,0.479616,1.598721,0.000000,3.597122,0.000000,1.438849,0.000000,7.354117,2.797762,9.832134
792,999,18.854839,3.191128,0.049650,0.440451,1.786400,0.613418,0.644549,0.618887,0.017351,...,0.000000,0.000000,0.000000,0.000000,0.118765,0.000000,1.543943,0.000000,8.907364,1.306413


In [81]:
df_essentia_best_overall_features_mean.to_csv(get_pmemo_path('processed/features/essentia_best_overall_features_mean.csv'))

Normalisation

In [82]:
scaler = MinMaxScaler()

In [83]:
# Fit and transform the selected columns
df_essentia_best_overall_features_mean_normalised = pd.DataFrame(scaler.fit_transform(df_essentia_best_overall_features_mean), columns=df_essentia_best_overall_features_mean.columns)

df_essentia_best_overall_features_mean_normalised = df_essentia_best_overall_features_mean_normalised.drop('song_id', axis=1)
df_essentia_best_overall_features_mean_normalised.insert(0, column='song_id', value=song_ids)

df_essentia_best_overall_features_mean_normalised.to_csv(get_pmemo_path('processed/features/normalised_essentia_best_overall_features_mean.csv'))

df_essentia_best_overall_features_mean_normalised

,song_id,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.spectral_energy.mean,tonal.chords_strength.mean,tonal.hpcp_entropy.mean,tonal.key_edma.strength,tonal.key_temperley.strength,rhythm.beats_loudness_band_ratio.mean_0,rhythm.beats_loudness_band_ratio.mean_1,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,1,0.052158,0.184221,0.099522,0.267193,0.482615,0.533418,0.580442,0.423573,0.086520,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.089160,0.000000,0.223899,0.098657,0.288181
1,4,0.279947,0.464124,0.749851,0.304507,0.461919,0.163914,0.177109,0.721330,0.117329,...,0.016972,0.077021,0.402838,0.000000,0.028650,0.117746,0.000000,0.107674,0.000000,0.155779
2,5,0.458712,0.547222,0.415546,0.464434,0.609828,0.714130,0.656943,0.750001,0.099329,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.008505,0.007329,0.000000,0.000000,0.000000
3,6,0.136278,0.366674,0.219986,0.362823,0.392684,0.381464,0.326521,0.736078,0.113002,...,0.000000,0.063161,0.000000,0.000000,0.000000,0.000000,0.000000,0.051213,0.000000,0.000000
4,7,0.041240,0.166304,0.159394,0.445732,0.675341,0.746939,0.786479,0.814618,0.090403,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.165104,0.096976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.047381,0.160340,0.145975,0.555560,0.580548,0.612516,0.664456,0.498154,0.420579,...,0.000000,0.000000,0.000000,0.031499,0.000000,0.000000,0.000000,0.013240,0.156009,0.863862
790,996,0.098750,0.201650,0.141929,0.552154,0.660622,0.638703,0.667402,0.145396,0.363266,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044460,0.016906
791,997,0.037444,0.172416,0.154091,0.144097,0.571115,0.554702,0.589613,0.489563,0.144911,...,0.008208,0.025688,0.000000,0.053602,0.000000,0.027702,0.000000,0.149709,0.035320,0.149458
792,999,0.051588,0.204707,0.188049,0.136955,0.490124,0.476657,0.523064,0.646732,0.019682,...,0.000000,0.000000,0.000000,0.000000,0.005146,0.000000,0.051231,0.000000,0.112450,0.019859


Standardisation

In [84]:
scaler = StandardScaler()

In [85]:
# Fit and transform the selected columns
df_essentia_best_overall_features_mean_standardised = pd.DataFrame(scaler.fit_transform(df_essentia_best_overall_features_mean), columns=df_essentia_best_overall_features_mean.columns)

df_essentia_best_overall_features_mean_standardised = df_essentia_best_overall_features_mean_standardised.drop('song_id', axis=1)
df_essentia_best_overall_features_mean_standardised.insert(0, column='song_id', value=song_ids)

df_essentia_best_overall_features_mean_standardised.to_csv(get_pmemo_path('processed/features/standardised_essentia_best_overall_features_mean.csv'))

df_essentia_best_overall_features_mean_standardised

,song_id,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.spectral_energy.mean,tonal.chords_strength.mean,tonal.hpcp_entropy.mean,tonal.key_edma.strength,tonal.key_temperley.strength,rhythm.beats_loudness_band_ratio.mean_0,rhythm.beats_loudness_band_ratio.mean_1,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,1,-0.282657,-0.162637,-0.648432,-0.964074,-0.686092,-0.464422,-0.220703,-0.474090,-0.739806,...,-0.197985,-0.308590,-0.247258,-0.449001,-0.271092,0.351076,-0.281493,1.028335,-0.031496,0.669736
1,4,1.944965,2.230816,5.895596,-0.694040,-0.868353,-2.500184,-2.338974,1.042443,-0.513609,...,0.006168,0.556358,4.083166,-0.449001,-0.008096,0.582452,-0.281493,0.217587,-0.840447,-0.056652
2,5,3.693159,2.941383,2.531610,0.463292,0.434211,0.531201,0.181074,1.188472,-0.645763,...,-0.197985,-0.308590,-0.247258,-0.449001,-0.271092,-0.301759,-0.198157,-0.533513,-0.840447,-0.911290
3,6,0.539981,1.397518,0.563754,-0.272032,-1.478070,-1.301602,-1.554277,1.117558,-0.545375,...,-0.197985,0.400710,-0.247258,-0.449001,-0.271092,-0.370599,-0.281493,-0.176268,-0.840447,-0.911290
4,7,-0.389423,-0.315849,-0.045958,0.327959,1.011148,0.711966,0.861390,1.517581,-0.711294,...,-0.197985,-0.308590,-0.247258,-0.449001,-0.271092,-0.370599,-0.281493,0.463013,0.513347,-0.379258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,-0.329374,-0.366843,-0.180989,1.122742,0.176350,-0.028631,0.220533,-0.094233,1.712845,...,-0.197985,-0.308590,-0.247258,-0.117204,-0.271092,-0.370599,-0.281493,-0.441152,0.438772,3.828050
790,996,0.172986,-0.013606,-0.221701,1.098095,0.881529,0.115642,0.236004,-1.890900,1.292057,...,-0.197985,-0.308590,-0.247258,-0.449001,-0.271092,-0.370599,-0.281493,-0.533513,-0.475888,-0.818542
791,997,-0.426545,-0.263585,-0.099324,-1.854877,0.093285,-0.347157,-0.172535,-0.137992,-0.311104,...,-0.099254,-0.020107,-0.247258,0.115623,-0.271092,-0.146373,-0.281493,0.510808,-0.550835,-0.091328
792,999,-0.288232,0.012533,0.242381,-1.906557,-0.619966,-0.777138,-0.522047,0.662503,-1.230531,...,-0.197985,-0.308590,-0.247258,-0.449001,-0.223849,-0.370599,0.301045,-0.533513,0.081605,-0.802341


In [86]:
df_essentia_best_arousal_features

,song_id,lowlevel.average_loudness,lowlevel.barkbands_spread.dmean,lowlevel.barkbands_spread.dmean2,lowlevel.barkbands_spread.dvar,lowlevel.barkbands_spread.dvar2,lowlevel.barkbands_spread.max,lowlevel.barkbands_spread.mean,lowlevel.barkbands_spread.median,lowlevel.barkbands_spread.min,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,1,0.970365,9.962052,14.903063,160.617004,331.039124,112.633751,30.026752,25.740023,0.681941,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.630969,0.000000,10.998553,7.814761,18.958033
1,4,0.938222,4.796559,7.069628,64.340286,119.074539,79.542397,8.677779,4.292209,0.278884,...,0.991735,4.793388,25.785124,0.000000,0.661157,6.115703,0.000000,5.289256,0.000000,10.247934
2,5,0.930816,8.162500,12.375443,168.968216,342.220551,117.832550,13.578498,5.963892,0.096577,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.441745,0.220872,0.000000,0.000000,0.000000
3,6,0.954429,8.911726,13.763487,122.484230,282.592804,78.847557,20.610300,16.304569,0.695686,...,0.000000,3.930818,0.000000,0.000000,0.000000,0.000000,0.000000,2.515723,0.000000,0.000000
4,7,0.970629,7.519822,11.455831,102.278740,221.926697,97.466393,23.933802,18.153156,0.721484,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.017544,13.078150,6.379585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.900841,6.240107,9.745099,77.798691,181.858902,117.897911,23.742342,21.268417,0.768844,...,0.000000,0.000000,0.000000,2.113821,0.000000,0.000000,0.000000,0.650406,12.357723,56.829270
790,996,0.923707,8.706299,13.284878,103.574509,212.724503,98.878693,29.687521,27.660221,0.308236,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.521780,1.112141
791,997,0.913454,8.977397,13.752587,112.814796,247.585236,128.250137,39.045204,36.508698,1.025961,...,0.479616,1.598721,0.000000,3.597122,0.000000,1.438849,0.000000,7.354117,2.797762,9.832134
792,999,0.955402,9.373850,14.096522,99.586502,224.760956,120.191139,44.180817,44.341572,3.119685,...,0.000000,0.000000,0.000000,0.000000,0.118765,0.000000,1.543943,0.000000,8.907364,1.306413


In [87]:
col_names = df_essentia_best_arousal_features.columns.to_list()
feature_mean_cols = [col for col in col_names if 'dmean' not in col and 'dmean2' not in col and 'dvar' not in col and 'dvar2' not in col and 'max' not in col and 'median' not in col and 'min' not in col and 'stdev' not in col and 'var' not in col]
print(feature_mean_cols)

['song_id', 'lowlevel.average_loudness', 'lowlevel.barkbands_spread.mean', 'lowlevel.melbands_crest.mean', 'lowlevel.melbands_flatness_db.mean', 'lowlevel.melbands_kurtosis.mean', 'lowlevel.melbands_skewness.mean', 'lowlevel.melbands_spread.mean', 'lowlevel.spectral_energy.mean', 'lowlevel.spectral_entropy.mean', 'lowlevel.spectral_flux.mean', 'lowlevel.spectral_kurtosis.mean', 'lowlevel.spectral_rolloff.mean', 'lowlevel.spectral_skewness.mean', 'rhythm.bpm_histogram_first_peak_bpm', 'rhythm.bpm_histogram_first_peak_weight', 'rhythm.bpm_histogram_second_peak_bpm', 'rhythm.bpm_histogram_second_peak_spread', 'rhythm.bpm_histogram_second_peak_weight', 'rhythm.danceability', 'rhythm.onset_rate', 'tonal.chords_strength.mean', 'tonal.hpcp_entropy.mean', 'tonal.key_edma.strength', 'tonal.key_temperley.strength', 'rhythm.beats_loudness_band_ratio.mean_0', 'rhythm.beats_loudness_band_ratio.mean_1', 'rhythm.beats_loudness_band_ratio.mean_2', 'rhythm.beats_loudness_band_ratio.mean_3', 'rhythm.bea

In [88]:
df_essentia_best_arousal_features_mean = df_essentia_best_arousal_features[feature_mean_cols]
df_essentia_best_arousal_features_mean

,song_id,lowlevel.average_loudness,lowlevel.barkbands_spread.mean,lowlevel.melbands_crest.mean,lowlevel.melbands_flatness_db.mean,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.melbands_spread.mean,lowlevel.spectral_energy.mean,lowlevel.spectral_entropy.mean,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,1,0.970365,30.026752,21.315563,0.210012,19.044914,2.916830,34.354786,0.030159,7.647935,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.630969,0.000000,10.998553,7.814761,18.958033
1,4,0.938222,8.677779,30.797611,0.372297,95.002518,6.664732,8.631808,0.173342,7.264801,...,0.991735,4.793388,25.785124,0.000000,0.661157,6.115703,0.000000,5.289256,0.000000,10.247934
2,5,0.930816,13.578498,26.574856,0.361869,154.612579,7.777407,15.093005,0.099738,7.315362,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.441745,0.220872,0.000000,0.000000,0.000000
3,6,0.954429,20.610300,28.322435,0.300273,47.095280,5.359872,16.303146,0.056681,7.528450,...,0.000000,3.930818,0.000000,0.000000,0.000000,0.000000,0.000000,2.515723,0.000000,0.000000
4,7,0.970629,23.933802,13.910563,0.183233,15.404405,2.676914,29.061829,0.043341,7.649847,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.017544,13.078150,6.379585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.900841,23.742342,16.773016,0.198606,17.451988,2.597063,29.571623,0.040386,7.658952,...,0.000000,0.000000,0.000000,2.113821,0.000000,0.000000,0.000000,0.650406,12.357723,56.829270
790,996,0.923707,29.687521,14.956428,0.155688,34.581482,3.150197,34.261261,0.039496,8.051765,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.521780,1.112141
791,997,0.913454,39.045204,18.180822,0.169300,14.138631,2.758755,38.427273,0.042173,8.011445,...,0.479616,1.598721,0.000000,3.597122,0.000000,1.438849,0.000000,7.354117,2.797762,9.832134
792,999,0.955402,44.180817,19.358309,0.162716,18.854839,3.191128,36.419331,0.049650,8.179975,...,0.000000,0.000000,0.000000,0.000000,0.118765,0.000000,1.543943,0.000000,8.907364,1.306413


In [89]:
df_essentia_best_arousal_features_mean.to_csv(get_pmemo_path('processed/features/essentia_best_arousal_features_mean.csv'))

Normalisation

In [90]:
scaler = MinMaxScaler()

In [91]:
# Fit and transform the selected columns
df_essentia_best_arousal_features_mean_normalised = pd.DataFrame(scaler.fit_transform(df_essentia_best_arousal_features_mean), columns=df_essentia_best_arousal_features_mean.columns)

df_essentia_best_arousal_features_mean_normalised = df_essentia_best_arousal_features_mean_normalised.drop('song_id', axis=1)
df_essentia_best_arousal_features_mean_normalised.insert(0, column='song_id', value=song_ids)

df_essentia_best_arousal_features_mean_normalised.to_csv(get_pmemo_path('processed/features/normalised_essentia_best_arousal_features_mean.csv'))

df_essentia_best_arousal_features_mean_normalised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_spread.mean,lowlevel.melbands_crest.mean,lowlevel.melbands_flatness_db.mean,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.melbands_spread.mean,lowlevel.spectral_energy.mean,lowlevel.spectral_entropy.mean,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,1,0.984985,0.583628,0.466983,0.314570,0.052158,0.184221,0.548723,0.099522,0.763577,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.089160,0.000000,0.223899,0.098657,0.288181
1,4,0.952336,0.082020,0.811687,0.690897,0.279947,0.464124,0.075821,0.749851,0.593323,...,0.016972,0.077021,0.402838,0.000000,0.028650,0.117746,0.000000,0.107674,0.000000,0.155779
2,5,0.944813,0.197166,0.658175,0.666715,0.458712,0.547222,0.194606,0.415546,0.615791,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.008505,0.007329,0.000000,0.000000,0.000000
3,6,0.968798,0.362382,0.721706,0.523877,0.136278,0.366674,0.216854,0.219986,0.710481,...,0.000000,0.063161,0.000000,0.000000,0.000000,0.000000,0.000000,0.051213,0.000000,0.000000
4,7,0.985253,0.440470,0.197786,0.252470,0.041240,0.166304,0.451415,0.159394,0.764426,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.165104,0.096976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.914366,0.435971,0.301846,0.288119,0.047381,0.160340,0.460787,0.145975,0.768472,...,0.000000,0.000000,0.000000,0.031499,0.000000,0.000000,0.000000,0.013240,0.156009,0.863862
790,996,0.937592,0.575657,0.235807,0.188595,0.098750,0.201650,0.547003,0.141929,0.943027,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044460,0.016906
791,997,0.927177,0.795522,0.353024,0.220160,0.037444,0.172416,0.623593,0.154091,0.925110,...,0.008208,0.025688,0.000000,0.053602,0.000000,0.027702,0.000000,0.149709,0.035320,0.149458
792,999,0.969786,0.916186,0.395830,0.204894,0.051588,0.204707,0.586678,0.188049,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.005146,0.000000,0.051231,0.000000,0.112450,0.019859


Standardisation

In [92]:
scaler = StandardScaler()

In [93]:
# Fit and transform the selected columns
df_essentia_best_arousal_features_mean_standardised = pd.DataFrame(scaler.fit_transform(df_essentia_best_arousal_features_mean), columns=df_essentia_best_arousal_features_mean.columns)

df_essentia_best_arousal_features_mean_standardised = df_essentia_best_arousal_features_mean_standardised.drop('song_id', axis=1)
df_essentia_best_arousal_features_mean_standardised.insert(0, column='song_id', value=song_ids)

df_essentia_best_arousal_features_mean_standardised.to_csv(get_pmemo_path('processed/features/standardised_essentia_best_arousal_features_mean.csv'))

df_essentia_best_arousal_features_mean_standardised

,song_id,lowlevel.average_loudness,lowlevel.barkbands_spread.mean,lowlevel.melbands_crest.mean,lowlevel.melbands_flatness_db.mean,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.melbands_spread.mean,lowlevel.spectral_energy.mean,lowlevel.spectral_entropy.mean,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,1,0.422535,0.929736,0.778042,-0.241194,-0.282657,-0.162637,1.128827,-0.648432,0.322213,...,-0.197985,-0.308590,-0.247258,-0.449001,-0.271092,0.351076,-0.281493,1.028335,-0.031496,0.669736
1,4,0.117155,-1.835937,2.690830,2.182449,1.944965,2.230816,-1.731936,5.895596,-0.688525,...,0.006168,0.556358,4.083166,-0.449001,-0.008096,0.582452,-0.281493,0.217587,-0.840447,-0.056652
2,5,0.046795,-1.201069,1.838985,2.026710,3.693159,2.941383,-1.013358,2.531610,-0.555141,...,-0.197985,-0.308590,-0.247258,-0.449001,-0.271092,-0.301759,-0.198157,-0.533513,-0.840447,-0.911290
3,6,0.271131,-0.290127,2.191519,1.106797,0.539981,1.397518,-0.878773,0.563754,0.007000,...,-0.197985,0.400710,-0.247258,-0.449001,-0.271092,-0.370599,-0.281493,-0.176268,-0.840447,-0.911290
4,7,0.425040,0.140419,-0.715749,-0.641130,-0.389423,-0.315849,0.540175,-0.045958,0.327256,...,-0.197985,-0.308590,-0.247258,-0.449001,-0.271092,-0.370599,-0.281493,0.463013,0.513347,-0.379258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,-0.237989,0.115616,-0.138314,-0.411544,-0.329374,-0.366843,0.596871,-0.180989,0.351276,...,-0.197985,-0.308590,-0.247258,-0.117204,-0.271092,-0.370599,-0.281493,-0.441152,0.438772,3.828050
790,996,-0.020750,0.885790,-0.504770,-1.052503,0.172986,-0.013606,1.118426,-0.221701,1.387548,...,-0.197985,-0.308590,-0.247258,-0.449001,-0.271092,-0.370599,-0.281493,-0.533513,-0.475888,-0.818542
791,997,-0.118164,2.098041,0.145679,-0.849214,-0.426545,-0.263585,1.581746,-0.099324,1.281179,...,-0.099254,-0.020107,-0.247258,0.115623,-0.271092,-0.146373,-0.281493,0.510808,-0.550835,-0.091328
792,999,0.280373,2.763339,0.383210,-0.947532,-0.288232,0.012533,1.358434,0.242381,1.725773,...,-0.197985,-0.308590,-0.247258,-0.449001,-0.223849,-0.370599,0.301045,-0.533513,0.081605,-0.802341


In [94]:
df_essentia_best_valence_features

,song_id,lowlevel.melbands_kurtosis.dmean,lowlevel.melbands_kurtosis.dmean2,lowlevel.melbands_kurtosis.dvar,lowlevel.melbands_kurtosis.dvar2,lowlevel.melbands_kurtosis.max,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_kurtosis.median,lowlevel.melbands_kurtosis.min,lowlevel.melbands_kurtosis.stdev,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,1,12.878959,19.579144,1312.717041,3060.723389,639.860474,19.044914,5.928576,-1.777708,44.479679,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.630969,0.000000,10.998553,7.814761,18.958033
1,4,47.323818,71.870140,7632.717773,15035.108398,1229.549561,95.002518,40.996933,-1.586174,143.324692,...,0.991735,4.793388,25.785124,0.000000,0.661157,6.115703,0.000000,5.289256,0.000000,10.247934
2,5,100.432968,153.566513,32880.425781,74328.078125,2090.814209,154.612579,60.239502,-1.944383,246.793365,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.441745,0.220872,0.000000,0.000000,0.000000
3,6,23.244177,35.301132,1514.325195,3573.641113,353.348297,47.095280,28.833210,-1.731773,55.418545,...,0.000000,3.930818,0.000000,0.000000,0.000000,0.000000,0.000000,2.515723,0.000000,0.000000
4,7,11.443307,17.701553,695.618958,1591.793823,481.152069,15.404405,7.509385,-1.873933,27.474340,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.017544,13.078150,6.379585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,9.660586,15.619761,918.862549,2431.855225,941.064697,17.451988,7.082492,-1.941114,33.566727,...,0.000000,0.000000,0.000000,2.113821,0.000000,0.000000,0.000000,0.650406,12.357723,56.829270
790,996,19.273998,26.486744,3824.638184,7618.275879,1317.343872,34.581482,5.004123,-1.680010,104.511772,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.521780,1.112141
791,997,8.544683,13.374214,346.121216,893.072449,364.284393,14.138631,6.815628,-1.910969,21.804148,...,0.479616,1.598721,0.000000,3.597122,0.000000,1.438849,0.000000,7.354117,2.797762,9.832134
792,999,9.426803,14.378345,278.683746,670.824463,287.886932,18.854839,8.583271,-1.448540,28.539162,...,0.000000,0.000000,0.000000,0.000000,0.118765,0.000000,1.543943,0.000000,8.907364,1.306413


In [95]:
col_names = df_essentia_best_valence_features.columns.to_list()
feature_mean_cols = [col for col in col_names if 'dmean' not in col and 'dmean2' not in col and 'dvar' not in col and 'dvar2' not in col and 'max' not in col and 'median' not in col and 'min' not in col and 'stdev' not in col and 'var' not in col]
print(feature_mean_cols)

['song_id', 'lowlevel.melbands_kurtosis.mean', 'lowlevel.melbands_skewness.mean', 'lowlevel.spectral_energy.mean', 'lowlevel.zerocrossingrate.mean', 'rhythm.beats_loudness.mean', 'rhythm.onset_rate', 'tonal.chords_strength.mean', 'tonal.hpcp_entropy.mean', 'tonal.key_edma.strength', 'tonal.key_temperley.strength', 'lowlevel.gfcc.cov_0', 'lowlevel.gfcc.cov_1', 'lowlevel.gfcc.cov_2', 'lowlevel.gfcc.cov_3', 'lowlevel.gfcc.cov_4', 'lowlevel.gfcc.cov_5', 'lowlevel.gfcc.cov_6', 'lowlevel.gfcc.cov_7', 'lowlevel.gfcc.cov_8', 'lowlevel.gfcc.cov_9', 'lowlevel.gfcc.cov_10', 'lowlevel.gfcc.cov_11', 'lowlevel.gfcc.cov_12', 'lowlevel.gfcc.cov_13', 'lowlevel.gfcc.cov_14', 'lowlevel.gfcc.cov_15', 'lowlevel.gfcc.cov_16', 'lowlevel.gfcc.cov_17', 'lowlevel.gfcc.cov_18', 'lowlevel.gfcc.cov_19', 'lowlevel.gfcc.cov_20', 'lowlevel.gfcc.cov_21', 'lowlevel.gfcc.cov_22', 'lowlevel.gfcc.cov_23', 'lowlevel.gfcc.cov_24', 'lowlevel.gfcc.cov_25', 'lowlevel.gfcc.cov_26', 'lowlevel.gfcc.cov_27', 'lowlevel.gfcc.cov_28'

In [96]:
df_essentia_best_valence_features_mean = df_essentia_best_valence_features[feature_mean_cols]
df_essentia_best_valence_features_mean

,song_id,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.spectral_energy.mean,lowlevel.zerocrossingrate.mean,rhythm.beats_loudness.mean,rhythm.onset_rate,tonal.chords_strength.mean,tonal.hpcp_entropy.mean,tonal.key_edma.strength,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,1,19.044914,2.916830,0.030159,0.075053,0.049574,5.118037,0.483265,1.773851,0.647135,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.630969,0.000000,10.998553,7.814761,18.958033
1,4,95.002518,6.664732,0.173342,0.020421,0.216983,5.026791,0.495532,1.739260,0.427637,...,0.991735,4.793388,25.785124,0.000000,0.661157,6.115703,0.000000,5.289256,0.000000,10.247934
2,5,154.612579,7.777407,0.099738,0.038424,0.148369,5.317868,0.548106,1.986469,0.754484,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.441745,0.220872,0.000000,0.000000,0.000000
3,6,47.095280,5.359872,0.056681,0.052518,0.073169,4.851120,0.514703,1.623544,0.556869,...,0.000000,3.930818,0.000000,0.000000,0.000000,0.000000,0.000000,2.515723,0.000000,0.000000
4,7,15.404405,2.676914,0.043341,0.079573,0.114386,3.750890,0.541958,2.095964,0.773975,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.017544,13.078150,6.379585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,17.451988,2.597063,0.040386,0.092792,0.066337,3.592518,0.578063,1.937530,0.694123,...,0.000000,0.000000,0.000000,2.113821,0.000000,0.000000,0.000000,0.650406,12.357723,56.829270
790,996,34.581482,3.150197,0.039496,0.107249,0.043886,4.314631,0.576943,2.071364,0.709678,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.521780,1.112141
791,997,14.138631,2.758755,0.042173,0.105412,0.054690,3.410177,0.442799,1.921766,0.659779,...,0.479616,1.598721,0.000000,3.597122,0.000000,1.438849,0.000000,7.354117,2.797762,9.832134
792,999,18.854839,3.191128,0.049650,0.111079,0.073621,2.894145,0.440451,1.786400,0.613418,...,0.000000,0.000000,0.000000,0.000000,0.118765,0.000000,1.543943,0.000000,8.907364,1.306413


In [97]:
df_essentia_best_valence_features_mean.to_csv(get_pmemo_path('processed/features/essentia_best_valence_features_mean.csv'))

Normalisation

In [98]:
scaler = MinMaxScaler()

In [99]:
# Fit and transform the selected columns
df_essentia_best_valence_features_mean_normalised = pd.DataFrame(scaler.fit_transform(df_essentia_best_valence_features_mean), columns=df_essentia_best_valence_features_mean.columns)

df_essentia_best_valence_features_mean_normalised = df_essentia_best_valence_features_mean_normalised.drop('song_id', axis=1)
df_essentia_best_valence_features_mean_normalised.insert(0, column='song_id', value=song_ids)

df_essentia_best_valence_features_mean_normalised.to_csv(get_pmemo_path('processed/features/normalised_essentia_best_valence_features_mean.csv'))

df_essentia_best_valence_features_mean_normalised

,song_id,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.spectral_energy.mean,lowlevel.zerocrossingrate.mean,rhythm.beats_loudness.mean,rhythm.onset_rate,tonal.chords_strength.mean,tonal.hpcp_entropy.mean,tonal.key_edma.strength,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,1,0.052158,0.184221,0.099522,0.416985,0.086404,0.800435,0.267193,0.482615,0.533418,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.089160,0.000000,0.223899,0.098657,0.288181
1,4,0.279947,0.464124,0.749851,0.050339,0.462009,0.785143,0.304507,0.461919,0.163914,...,0.016972,0.077021,0.402838,0.000000,0.028650,0.117746,0.000000,0.107674,0.000000,0.155779
2,5,0.458712,0.547222,0.415546,0.171160,0.308063,0.833923,0.464434,0.609828,0.714130,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.008505,0.007329,0.000000,0.000000,0.000000
3,6,0.136278,0.366674,0.219986,0.265749,0.139343,0.755704,0.362823,0.392684,0.381464,...,0.000000,0.063161,0.000000,0.000000,0.000000,0.000000,0.000000,0.051213,0.000000,0.000000
4,7,0.041240,0.166304,0.159394,0.447316,0.231818,0.571324,0.445732,0.675341,0.746939,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.165104,0.096976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,0.047381,0.160340,0.145975,0.536035,0.124013,0.544784,0.555560,0.580548,0.612516,...,0.000000,0.000000,0.000000,0.031499,0.000000,0.000000,0.000000,0.013240,0.156009,0.863862
790,996,0.098750,0.201650,0.141929,0.633059,0.073642,0.665797,0.552154,0.660622,0.638703,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044460,0.016906
791,997,0.037444,0.172416,0.154091,0.620730,0.097882,0.514226,0.144097,0.571115,0.554702,...,0.008208,0.025688,0.000000,0.053602,0.000000,0.027702,0.000000,0.149709,0.035320,0.149458
792,999,0.051588,0.204707,0.188049,0.658759,0.140357,0.427748,0.136955,0.490124,0.476657,...,0.000000,0.000000,0.000000,0.000000,0.005146,0.000000,0.051231,0.000000,0.112450,0.019859


Standardisation

In [100]:
scaler = StandardScaler()

In [101]:
# Fit and transform the selected columns
df_essentia_best_valence_features_mean_standardised = pd.DataFrame(scaler.fit_transform(df_essentia_best_valence_features_mean), columns=df_essentia_best_valence_features_mean.columns)

df_essentia_best_valence_features_mean_standardised = df_essentia_best_valence_features_mean_standardised.drop('song_id', axis=1)
df_essentia_best_valence_features_mean_standardised.insert(0, column='song_id', value=song_ids)

df_essentia_best_valence_features_mean_standardised.to_csv(get_pmemo_path('processed/features/standardised_essentia_best_valence_features_mean.csv'))

df_essentia_best_valence_features_mean_standardised

,song_id,lowlevel.melbands_kurtosis.mean,lowlevel.melbands_skewness.mean,lowlevel.spectral_energy.mean,lowlevel.zerocrossingrate.mean,rhythm.beats_loudness.mean,rhythm.onset_rate,tonal.chords_strength.mean,tonal.hpcp_entropy.mean,tonal.key_edma.strength,...,tonal.chords_histogram_14,tonal.chords_histogram_15,tonal.chords_histogram_16,tonal.chords_histogram_17,tonal.chords_histogram_18,tonal.chords_histogram_19,tonal.chords_histogram_20,tonal.chords_histogram_21,tonal.chords_histogram_22,tonal.chords_histogram_23
0,1,-0.282657,-0.162637,-0.648432,0.595077,-0.444325,1.289894,-0.964074,-0.686092,-0.464422,...,-0.197985,-0.308590,-0.247258,-0.449001,-0.271092,0.351076,-0.281493,1.028335,-0.031496,0.669736
1,4,1.944965,2.230816,5.895596,-2.018322,4.051903,1.190645,-0.694040,-0.868353,-2.500184,...,0.006168,0.556358,4.083166,-0.449001,-0.008096,0.582452,-0.281493,0.217587,-0.840447,-0.056652
2,5,3.693159,2.941383,2.531610,-1.157125,2.209071,1.507253,0.463292,0.434211,0.531201,...,-0.197985,-0.308590,-0.247258,-0.449001,-0.271092,-0.301759,-0.198157,-0.533513,-0.840447,-0.911290
3,6,0.539981,1.397518,0.563754,-0.482907,0.189379,0.999565,-0.272032,-1.478070,-1.301602,...,-0.197985,0.400710,-0.247258,-0.449001,-0.271092,-0.370599,-0.281493,-0.176268,-0.840447,-0.911290
4,7,-0.389423,-0.315849,-0.045958,0.811273,1.296371,-0.197167,0.327959,1.011148,0.711966,...,-0.197985,-0.308590,-0.247258,-0.449001,-0.271092,-0.370599,-0.281493,0.463013,0.513347,-0.379258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,993,-0.329374,-0.366843,-0.180989,1.443650,0.005877,-0.369430,1.122742,0.176350,-0.028631,...,-0.197985,-0.308590,-0.247258,-0.117204,-0.271092,-0.370599,-0.281493,-0.441152,0.438772,3.828050
790,996,0.172986,-0.013606,-0.221701,2.135221,-0.597100,0.416020,1.098095,0.881529,0.115642,...,-0.197985,-0.308590,-0.247258,-0.449001,-0.271092,-0.370599,-0.281493,-0.533513,-0.475888,-0.818542
791,997,-0.426545,-0.263585,-0.099324,2.047344,-0.306936,-0.567764,-1.854877,0.093285,-0.347157,...,-0.099254,-0.020107,-0.247258,0.115623,-0.271092,-0.146373,-0.281493,0.510808,-0.550835,-0.091328
792,999,-0.288232,0.012533,0.242381,2.318405,0.201522,-1.129059,-1.906557,-0.619966,-0.777138,...,-0.197985,-0.308590,-0.247258,-0.449001,-0.223849,-0.370599,0.301045,-0.533513,0.081605,-0.802341
